In [1]:
import os
import glob
import re
from dataclasses import dataclass, field
from typing import List, Optional
from bs4 import BeautifulSoup


# data classes for monster sections
@dataclass
class Trait:
    name: str
    text: str


@dataclass
class BonusAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class LegendaryAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class Action:
    name: str
    text: str
    category: Optional[str] = None     # attack, save, other

    # attack fields
    attack_bonus: Optional[int] = None
    damage_dice: Optional[str] = None
    damage_type: Optional[str] = None
    reach: Optional[str] = None
    range: Optional[str] = None

    # save fields
    save_ability: Optional[str] = None
    save_dc: Optional[int] = None

    usage: Optional[str] = None


@dataclass
class Monster:
    name: str

    # type & alignment
    mtype: Optional[str] = None
    size: Optional[str] = None
    creature_type: Optional[str] = None
    alignment: Optional[str] = None

    # core stats
    ac: Optional[int] = None
    initiative: Optional[int] = None
    hp: Optional[int] = None
    speed: Optional[str] = None
    skills: Optional[str] = None
    resistances: Optional[str] = None
    senses: Optional[str] = None
    languages: Optional[str] = None
    cr: Optional[str] = None
    pb: Optional[str] = None
    passive_perception: Optional[int] = None

    # abilities
    STR_score: Optional[str] = None
    STR_mod: Optional[str] = None
    STR_save: Optional[str] = None
    DEX_score: Optional[str] = None
    DEX_mod: Optional[str] = None
    DEX_save: Optional[str] = None
    CON_score: Optional[str] = None
    CON_mod: Optional[str] = None
    CON_save: Optional[str] = None
    INT_score: Optional[str] = None
    INT_mod: Optional[str] = None
    INT_save: Optional[str] = None
    WIS_score: Optional[str] = None
    WIS_mod: Optional[str] = None
    WIS_save: Optional[str] = None
    CHA_score: Optional[str] = None
    CHA_mod: Optional[str] = None
    CHA_save: Optional[str] = None

    # structured lists
    traits: List[Trait] = field(default_factory=list)
    actions: List[Action] = field(default_factory=list)
    bonus_actions: List[BonusAction] = field(default_factory=list)
    legendary_actions: List[LegendaryAction] = field(default_factory=list)


# parse the Traits section under the Traits header
def parse_traits(block):
    traits = []
    header = block.find("p", class_="monster-header", string=re.compile("Traits", re.I))
    if not header:
        return traits

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        text = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        traits.append(Trait(name=name, text=text))

    return traits


# parse the Actions section and classify each action
def parse_actions(block):
    actions = []
    header = block.find("p", class_="monster-header", string=re.compile("Actions", re.I))
    if not header:
        return actions

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect (2/Day) style usages in the action name
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the front to get the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        # classify the action type (attack, save, or other)
        if re.search(r"Attack Roll", txt):
            category = "attack"
        elif re.search(r"Saving Throw", txt):
            category = "save"
        else:
            category = "other"

        # extract attack-related fields
        m_hit = re.search(r"Attack(?: Roll)?:\s*([+-]\d+)", txt)
        m_dmg = re.search(r"\(([\dd+\-\s]+)\)\s*([A-Za-z]+) damage", txt)
        m_reach = re.search(r"reach\s+(.+?ft\.)", txt)
        m_range = re.search(r"range\s+(.+?ft\.)", txt)

        # extract save-related fields
        m_save = re.search(r"([A-Za-z]+)\s+Saving Throw:\s*DC\s*(\d+)", txt)

        actions.append(
            Action(
                name=name,
                text=body,
                category=category,
                usage=usage,
                attack_bonus=int(m_hit.group(1)) if m_hit else None,
                damage_dice=m_dmg.group(1).strip() if m_dmg else None,
                damage_type=m_dmg.group(2).lower() if m_dmg else None,
                reach=m_reach.group(1) if m_reach else None,
                range=m_range.group(1) if m_range else None,
                save_ability=m_save.group(1) if m_save else None,
                save_dc=int(m_save.group(2)) if m_save else None,
            )
        )

    return actions


# parse the Bonus Actions section
def parse_bonus_actions(block):
    bonus = []
    header = block.find("p", class_="monster-header", string=re.compile("Bonus Actions", re.I))
    if not header:
        return bonus

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect usage tags like (2/Day) in the name
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        bonus.append(BonusAction(name=name, text=body, usage=usage))

    return bonus


# parse the Legendary Actions section
def parse_legendary_actions(block):
    legs = []
    header = block.find("p", class_="monster-header", string=re.compile("Legendary Actions", re.I))
    if not header:
        return legs

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect usage tags if present
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        legs.append(LegendaryAction(name=name, text=body, usage=usage))

    return legs


# parse one HTML file into a list of Monster objects
def parse_monster_file(html: str) -> List[Monster]:
    soup = BeautifulSoup(html, "html.parser")
    monsters = []

    # loop over each stat-block in the HTML
    for block in soup.find_all("div", class_=lambda c: c and "stat-block" in c):

        # get the monster name from the heading
        header = block.find(["h3", "h4"], class_=re.compile("heading-anchor")) or \
                 block.find(["h3", "h4"])
        if not header:
            continue

        name = header.get_text(" ", strip=True)

        # get the type line right after the header
        type_p = header.find_next("p")
        mtype = type_p.get_text(" ", strip=True) if type_p else None

        size = creature_type = alignment = None
        if mtype:
            parts = mtype.split(",", 1)
            size_type = parts[0].strip()
            alignment = parts[1].strip() if len(parts) > 1 else None

            # split size/type with support for "Small or Medium"
            words = size_type.split()
            if len(words) >= 3 and words[1].lower() == "or":
                size = " ".join(words[:3])
                creature_type = " ".join(words[3:])
            else:
                size = words[0]
                creature_type = " ".join(words[1:])

        # collect all paragraph text in the stat block
        ps = block.find_all("p")
        text_lines = [p.get_text(" ", strip=True) for p in ps]

        ac = init = hp = None

        # parse AC, Initiative, and HP lines
        for t in text_lines:
            if t.startswith("AC"):
                m_ac = re.search(r"AC\s+(\d+)", t)
                ac = int(m_ac.group(1)) if m_ac else None
                m_init = re.search(r"Initiative\s*([+-]?\d+)", t)
                init = int(m_init.group(1)) if m_init else None

            if t.startswith("HP"):
                m_hp = re.search(r"HP\s+(\d+)", t)
                hp = int(m_hp.group(1)) if m_hp else None

        # helper to extract simple key/value lines (Speed, Skills, etc.)
        def extract_line(key: str):
            for t in text_lines:
                if t.startswith(key):
                    return re.sub(rf"^{key}\s*", "", t).strip()
            return None

        speed       = extract_line("Speed")
        skills      = extract_line("Skills")
        resistances = extract_line("Resistances")
        senses      = extract_line("Senses")
        languages   = extract_line("Languages")
        cr_text     = extract_line("CR")

        pb = None
        if cr_text:
            # parse PB from CR line
            m_pb = re.search(r"PB\s*([+-]\d+)", cr_text)
            pb = m_pb.group(1) if m_pb else None
            # parse numeric CR
            m_cr = re.search(r"^([0-9]+(?:/[0-9]+)?)", cr_text)
            if m_cr:
                cr_text = m_cr.group(1)

        # parse Passive Perception and clean the Senses string
        passive = None
        if senses:
            m_pass = re.search(r"Passive Perception\s*(\d+)", senses)
            if m_pass:
                passive = int(m_pass.group(1))
                senses = re.sub(r";?\s*Passive Perception\s*\d+", "", senses).strip(" ;")
                if senses == "":
                    senses = None

        # initialize ability fields
        ability_fields = {}
        for abbr in ("STR","DEX","CON","INT","WIS","CHA"):
            ability_fields[f"{abbr}_score"] = None
            ability_fields[f"{abbr}_mod"] = None
            ability_fields[f"{abbr}_save"] = None

        # parse the physical and mental ability tables
        for tbl_class in ("physical abilities-saves", "mental abilities-saves"):
            tbl = block.find("table", class_=tbl_class)
            if tbl and tbl.tbody:
                for row in tbl.tbody.find_all("tr"):
                    th = row.find("th")
                    if not th:
                        continue
                    abbr = th.get_text(" ", strip=True).upper()
                    if abbr not in ("STR","DEX","CON","INT","WIS","CHA"):
                        continue
                    cells = row.find_all("td")
                    if len(cells) >= 3:
                        ability_fields[f"{abbr}_score"] = cells[0].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_mod"]   = cells[1].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_save"]  = cells[2].get_text(" ", strip=True)

        # parse traits, actions, bonus actions, and legendary actions
        traits            = parse_traits(block)
        actions           = parse_actions(block)
        bonus_actions     = parse_bonus_actions(block)
        legendary_actions = parse_legendary_actions(block)

        # build the Monster object
        monsters.append(
            Monster(
                name=name,
                mtype=mtype,
                size=size,
                creature_type=creature_type,
                alignment=alignment,
                ac=ac,
                initiative=init,
                hp=hp,
                speed=speed,
                skills=skills,
                resistances=resistances,
                senses=senses,
                languages=languages,
                cr=cr_text,
                pb=pb,
                passive_perception=passive,
                traits=traits,
                actions=actions,
                bonus_actions=bonus_actions,
                legendary_actions=legendary_actions,
                **ability_fields
            )
        )

    return monsters


# print a readable stat block for a single monster
def pretty_print_monster(m: Monster):
    print("=" * 70)
    print(m.name)
    if m.size or m.creature_type:
        print(f"{m.size or ''} {m.creature_type or ''}, {m.alignment or ''}".strip())
    print("-" * 70)

    print(f"AC {m.ac or '—'} | HP {m.hp or '—'} | Initiative {m.initiative or '—'}")
    print(f"Speed: {m.speed or '—'}")
    if m.skills:
        print(f"Skills: {m.skills}")
    if m.resistances:
        print(f"Resistances: {m.resistances}")

    senses_line = m.senses or "—"
    if m.passive_perception:
        senses_line += f" (Passive Perception {m.passive_perception})"
    print(f"Senses: {senses_line}")

    print(f"Languages: {m.languages or '—'}")
    cr_disp = m.cr or "—"
    if m.pb:
        cr_disp += f" (PB {m.pb})"
    print(f"CR: {cr_disp}")

    print("\nAbility Scores:")
    def score(abbr):
        sc = getattr(m, f"{abbr}_score") or "—"
        md = getattr(m, f"{abbr}_mod") or "—"
        sv = getattr(m, f"{abbr}_save") or "—"
        return f"{abbr} {sc} ({md}) Save {sv}"

    print(f"{score('STR'):28} {score('DEX')}")
    print(f"{score('CON'):28} {score('INT')}")
    print(f"{score('WIS'):28} {score('CHA')}")

    if m.traits:
        print("\nTraits:")
        for t in m.traits:
            print(f"  • {t.name}: {t.text}")

    if m.actions:
        print("\nActions:")
        for a in m.actions:
            if a.usage:
                print(f"  • {a.name} ({a.usage}): {a.text}")
            else:
                print(f"  • {a.name}: {a.text}")

    if m.bonus_actions:
        print("\nBonus Actions:")
        for b in m.bonus_actions:
            if b.usage:
                print(f"  • {b.name} ({b.usage}): {b.text}")
            else:
                print(f"  • {b.name}: {b.text}")

    if m.legendary_actions:
        print("\nLegendary Actions:")
        for l in m.legendary_actions:
            if l.usage:
                print(f"  • {l.name} ({l.usage}): {l.text}")
            else:
                print(f"  • {l.name}: {l.text}")

    print("=" * 70)


# find a monster by name in the list (case-insensitive)
def get_monster_by_name(name: str, monsters: List[Monster]) -> Optional[Monster]:
    name = name.lower().strip()
    for m in monsters:
        if m.name.lower() == name:
            return m
    return None


# promptless helper to show a monster by name
def show_monster_by_name(monsters: List[Monster], name: str):
    m = get_monster_by_name(name, monsters)
    if m:
        pretty_print_monster(m)
    else:
        print("Monster not found.")


# main exec
# set this to the local/mounted path where your .txt stat block files live
# for example, in Colab after mounting Drive:
# folder = "/content/drive/MyDrive/dungeonsandanalysis/mm2024"
folder = "/content/drive/MyDrive/mm2024"  # TODO: adjust this to your actual folder

all_monsters: List[Monster] = []

for path in sorted(glob.glob(os.path.join(folder, "*.txt"))):
    with open(path, "r", encoding="utf-8") as f:
        html = f.read()
    all_monsters.extend(parse_monster_file(html))

print(f"Loaded {len(all_monsters)} monsters.")

# example usage:
# show_monster_by_name(all_monsters, "Aboleth")
# show_monster_by_name(all_monsters, "Aarakocra Aeromancer")


Loaded 0 monsters.


In [2]:
import re
import requests
from dataclasses import dataclass, field
from typing import List, Optional
from bs4 import BeautifulSoup


# data classes for monster sections
@dataclass
class Trait:
    name: str
    text: str


@dataclass
class BonusAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class LegendaryAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class Action:
    name: str
    text: str
    category: Optional[str] = None     # attack, save, other

    # attack fields
    attack_bonus: Optional[int] = None
    damage_dice: Optional[str] = None
    damage_type: Optional[str] = None
    reach: Optional[str] = None
    range: Optional[str] = None

    # save fields
    save_ability: Optional[str] = None
    save_dc: Optional[int] = None

    usage: Optional[str] = None


@dataclass
class Monster:
    name: str

    # type & alignment
    mtype: Optional[str] = None
    size: Optional[str] = None
    creature_type: Optional[str] = None
    alignment: Optional[str] = None

    # core stats
    ac: Optional[int] = None
    initiative: Optional[int] = None
    hp: Optional[int] = None
    speed: Optional[str] = None
    skills: Optional[str] = None
    resistances: Optional[str] = None
    senses: Optional[str] = None
    languages: Optional[str] = None
    cr: Optional[str] = None
    pb: Optional[str] = None
    passive_perception: Optional[int] = None

    # abilities
    STR_score: Optional[str] = None
    STR_mod: Optional[str] = None
    STR_save: Optional[str] = None
    DEX_score: Optional[str] = None
    DEX_mod: Optional[str] = None
    DEX_save: Optional[str] = None
    CON_score: Optional[str] = None
    CON_mod: Optional[str] = None
    CON_save: Optional[str] = None
    INT_score: Optional[str] = None
    INT_mod: Optional[str] = None
    INT_save: Optional[str] = None
    WIS_score: Optional[str] = None
    WIS_mod: Optional[str] = None
    WIS_save: Optional[str] = None
    CHA_score: Optional[str] = None
    CHA_mod: Optional[str] = None
    CHA_save: Optional[str] = None

    # structured lists
    traits: List[Trait] = field(default_factory=list)
    actions: List[Action] = field(default_factory=list)
    bonus_actions: List[BonusAction] = field(default_factory=list)
    legendary_actions: List[LegendaryAction] = field(default_factory=list)


# parse the Traits section under the Traits header
def parse_traits(block):
    traits = []
    header = block.find("p", class_="monster-header", string=re.compile("Traits", re.I))
    if not header:
        return traits

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        text = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        traits.append(Trait(name=name, text=text))

    return traits


# parse the Actions section and classify each action
def parse_actions(block):
    actions = []
    header = block.find("p", class_="monster-header", string=re.compile("Actions", re.I))
    if not header:
        return actions

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect (2/Day) style usages in the action name
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the front to get the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        # classify the action type (attack, save, or other)
        if re.search(r"Attack Roll", txt):
            category = "attack"
        elif re.search(r"Saving Throw", txt):
            category = "save"
        else:
            category = "other"

        # extract attack-related fields
        m_hit = re.search(r"Attack(?: Roll)?:\s*([+-]\d+)", txt)
        m_dmg = re.search(r"\(([\dd+\-\s]+)\)\s*([A-Za-z]+) damage", txt)
        m_reach = re.search(r"reach\s+(.+?ft\.)", txt)
        m_range = re.search(r"range\s+(.+?ft\.)", txt)

        # extract save-related fields
        m_save = re.search(r"([A-Za-z]+)\s+Saving Throw:\s*DC\s*(\d+)", txt)

        actions.append(
            Action(
                name=name,
                text=body,
                category=category,
                usage=usage,
                attack_bonus=int(m_hit.group(1)) if m_hit else None,
                damage_dice=m_dmg.group(1).strip() if m_dmg else None,
                damage_type=m_dmg.group(2).lower() if m_dmg else None,
                reach=m_reach.group(1) if m_reach else None,
                range=m_range.group(1) if m_range else None,
                save_ability=m_save.group(1) if m_save else None,
                save_dc=int(m_save.group(2)) if m_save else None,
            )
        )

    return actions


# parse the Bonus Actions section
def parse_bonus_actions(block):
    bonus = []
    header = block.find("p", class_="monster-header", string=re.compile("Bonus Actions", re.I))
    if not header:
        return bonus

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect usage tags like (2/Day) in the name
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        bonus.append(BonusAction(name=name, text=body, usage=usage))

    return bonus


# parse the Legendary Actions section
def parse_legendary_actions(block):
    legs = []
    header = block.find("p", class_="monster-header", string=re.compile("Legendary Actions", re.I))
    if not header:
        return legs

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect usage tags if present
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        legs.append(LegendaryAction(name=name, text=body, usage=usage))

    return legs


# parse one HTML file into a list of Monster objects
def parse_monster_file(html: str) -> List[Monster]:
    soup = BeautifulSoup(html, "html.parser")
    monsters = []

    # loop over each stat-block in the HTML
    for block in soup.find_all("div", class_=lambda c: c and "stat-block" in c):

        # get the monster name from the heading
        header = block.find(["h3", "h4"], class_=re.compile("heading-anchor")) or \
                 block.find(["h3", "h4"])
        if not header:
            continue

        name = header.get_text(" ", strip=True)

        # get the type line right after the header
        type_p = header.find_next("p")
        mtype = type_p.get_text(" ", strip=True) if type_p else None

        size = creature_type = alignment = None
        if mtype:
            parts = mtype.split(",", 1)
            size_type = parts[0].strip()
            alignment = parts[1].strip() if len(parts) > 1 else None

            # split size/type with support for "Small or Medium"
            words = size_type.split()
            if len(words) >= 3 and words[1].lower() == "or":
                size = " ".join(words[:3])
                creature_type = " ".join(words[3:])
            else:
                size = words[0]
                creature_type = " ".join(words[1:])

        # collect all paragraph text in the stat block
        ps = block.find_all("p")
        text_lines = [p.get_text(" ", strip=True) for p in ps]

        ac = init = hp = None

        # parse AC, Initiative, and HP lines
        for t in text_lines:
            if t.startswith("AC"):
                m_ac = re.search(r"AC\s+(\d+)", t)
                ac = int(m_ac.group(1)) if m_ac else None
                m_init = re.search(r"Initiative\s*([+-]?\d+)", t)
                init = int(m_init.group(1)) if m_init else None

            if t.startswith("HP"):
                m_hp = re.search(r"HP\s+(\d+)", t)
                hp = int(m_hp.group(1)) if m_hp else None

        # helper to extract simple key/value lines (Speed, Skills, etc.)
        def extract_line(key: str):
            for t in text_lines:
                if t.startswith(key):
                    return re.sub(rf"^{key}\s*", "", t).strip()
            return None

        speed       = extract_line("Speed")
        skills      = extract_line("Skills")
        resistances = extract_line("Resistances")
        senses      = extract_line("Senses")
        languages   = extract_line("Languages")
        cr_text     = extract_line("CR")

        pb = None
        if cr_text:
            # parse PB from CR line
            m_pb = re.search(r"PB\s*([+-]\d+)", cr_text)
            pb = m_pb.group(1) if m_pb else None
            # parse numeric CR
            m_cr = re.search(r"^([0-9]+(?:/[0-9]+)?)", cr_text)
            if m_cr:
                cr_text = m_cr.group(1)

        # parse Passive Perception and clean the Senses string
        passive = None
        if senses:
            m_pass = re.search(r"Passive Perception\s*(\d+)", senses)
            if m_pass:
                passive = int(m_pass.group(1))
                senses = re.sub(r";?\s*Passive Perception\s*\d+", "", senses).strip(" ;")
                if senses == "":
                    senses = None

        # initialize ability fields
        ability_fields = {}
        for abbr in ("STR","DEX","CON","INT","WIS","CHA"):
            ability_fields[f"{abbr}_score"] = None
            ability_fields[f"{abbr}_mod"] = None
            ability_fields[f"{abbr}_save"] = None

        # parse the physical and mental ability tables
        for tbl_class in ("physical abilities-saves", "mental abilities-saves"):
            tbl = block.find("table", class_=tbl_class)
            if tbl and tbl.tbody:
                for row in tbl.tbody.find_all("tr"):
                    th = row.find("th")
                    if not th:
                        continue
                    abbr = th.get_text(" ", strip=True).upper()
                    if abbr not in ("STR","DEX","CON","INT","WIS","CHA"):
                        continue
                    cells = row.find_all("td")
                    if len(cells) >= 3:
                        ability_fields[f"{abbr}_score"] = cells[0].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_mod"]   = cells[1].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_save"]  = cells[2].get_text(" ", strip=True)

        # parse traits, actions, bonus actions, and legendary actions
        traits            = parse_traits(block)
        actions           = parse_actions(block)
        bonus_actions     = parse_bonus_actions(block)
        legendary_actions = parse_legendary_actions(block)

        # build the Monster object
        monsters.append(
            Monster(
                name=name,
                mtype=mtype,
                size=size,
                creature_type=creature_type,
                alignment=alignment,
                ac=ac,
                initiative=init,
                hp=hp,
                speed=speed,
                skills=skills,
                resistances=resistances,
                senses=senses,
                languages=languages,
                cr=cr_text,
                pb=pb,
                passive_perception=passive,
                traits=traits,
                actions=actions,
                bonus_actions=bonus_actions,
                legendary_actions=legendary_actions,
                **ability_fields
            )
        )

    return monsters


# print a readable stat block for a single monster
def pretty_print_monster(m: Monster):
    print("=" * 70)
    print(m.name)
    if m.size or m.creature_type:
        print(f"{m.size or ''} {m.creature_type or ''}, {m.alignment or ''}".strip())
    print("-" * 70)

    print(f"AC {m.ac or '—'} | HP {m.hp or '—'} | Initiative {m.initiative or '—'}")
    print(f"Speed: {m.speed or '—'}")
    if m.skills:
        print(f"Skills: {m.skills}")
    if m.resistances:
        print(f"Resistances: {m.resistances}")

    senses_line = m.senses or "—"
    if m.passive_perception:
        senses_line += f" (Passive Perception {m.passive_perception})"
    print(f"Senses: {senses_line}")

    print(f"Languages: {m.languages or '—'}")
    cr_disp = m.cr or "—"
    if m.pb:
        cr_disp += f" (PB {m.pb})"
    print(f"CR: {cr_disp}")

    print("\nAbility Scores:")
    def score(abbr):
        sc = getattr(m, f"{abbr}_score") or "—"
        md = getattr(m, f"{abbr}_mod") or "—"
        sv = getattr(m, f"{abbr}_save") or "—"
        return f"{abbr} {sc} ({md}) Save {sv}"

    print(f"{score('STR'):28} {score('DEX')}")
    print(f"{score('CON'):28} {score('INT')}")
    print(f"{score('WIS'):28} {score('CHA')}")

    if m.traits:
        print("\nTraits:")
        for t in m.traits:
            print(f"  • {t.name}: {t.text}")

    if m.actions:
        print("\nActions:")
        for a in m.actions:
            if a.usage:
                print(f"  • {a.name} ({a.usage}): {a.text}")
            else:
                print(f"  • {a.name}: {a.text}")

    if m.bonus_actions:
        print("\nBonus Actions:")
        for b in m.bonus_actions:
            if b.usage:
                print(f"  • {b.name} ({b.usage}): {b.text}")
            else:
                print(f"  • {b.name}: {b.text}")

    if m.legendary_actions:
        print("\nLegendary Actions:")
        for l in m.legendary_actions:
            if l.usage:
                print(f"  • {l.name} ({l.usage}): {l.text}")
            else:
                print(f"  • {l.name}: {l.text}")

    print("=" * 70)


# find a monster by name in the list (case-insensitive)
def get_monster_by_name(name: str, monsters: List[Monster]) -> Optional[Monster]:
    name = name.lower().strip()
    for m in monsters:
        if m.name.lower() == name:
            return m
    return None


# promptless helper to show a monster by name
def show_monster_by_name(monsters: List[Monster], name: str):
    m = get_monster_by_name(name, monsters)
    if m:
        pretty_print_monster(m)
    else:
        print("Monster not found.")


# main exec – load from OneDrive URLs instead of local files
MONSTER_FILE_URLS = [
    # your first OneDrive file link:
    "https://1drv.ms/t/c/b4ffc266ad10f68d/IQAIPGx4pqgyQ4Wi3_0CTRMtASS9Jlqm0YNEa3E792uW0ZI?e=z7LEUa",
    # add more file links here as you create them
]

all_monsters: List[Monster] = []

for url in MONSTER_FILE_URLS:
    print(f"Downloading: {url}")
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"  -> failed with status {resp.status_code}")
        continue
    html = resp.text
    monsters_from_file = parse_monster_file(html)
    print(f"  -> parsed {len(monsters_from_file)} monsters")
    all_monsters.extend(monsters_from_file)

print(f"Total monsters loaded: {len(all_monsters)}")

# example usage (uncomment after first run to test)
# show_monster_by_name(all_monsters, "Aboleth")


Downloading: https://1drv.ms/t/c/b4ffc266ad10f68d/IQAIPGx4pqgyQ4Wi3_0CTRMtASS9Jlqm0YNEa3E792uW0ZI?e=z7LEUa
  -> parsed 0 monsters
Total monsters loaded: 0


In [6]:
import re
import requests
from dataclasses import dataclass, field
from typing import List, Optional
from bs4 import BeautifulSoup


# data classes for monster sections
@dataclass
class Trait:
    name: str
    text: str


@dataclass
class BonusAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class LegendaryAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class Action:
    name: str
    text: str
    category: Optional[str] = None     # attack, save, other

    # attack fields
    attack_bonus: Optional[int] = None
    damage_dice: Optional[str] = None
    damage_type: Optional[str] = None
    reach: Optional[str] = None
    range: Optional[str] = None

    # save fields
    save_ability: Optional[str] = None
    save_dc: Optional[int] = None

    usage: Optional[str] = None


@dataclass
class Monster:
    name: str

    # type & alignment
    mtype: Optional[str] = None
    size: Optional[str] = None
    creature_type: Optional[str] = None
    alignment: Optional[str] = None

    # core stats
    ac: Optional[int] = None
    initiative: Optional[int] = None
    hp: Optional[int] = None
    speed: Optional[str] = None
    skills: Optional[str] = None
    resistances: Optional[str] = None
    senses: Optional[str] = None
    languages: Optional[str] = None
    cr: Optional[str] = None
    pb: Optional[str] = None
    passive_perception: Optional[int] = None

    # abilities
    STR_score: Optional[str] = None
    STR_mod: Optional[str] = None
    STR_save: Optional[str] = None
    DEX_score: Optional[str] = None
    DEX_mod: Optional[str] = None
    DEX_save: Optional[str] = None
    CON_score: Optional[str] = None
    CON_mod: Optional[str] = None
    CON_save: Optional[str] = None
    INT_score: Optional[str] = None
    INT_mod: Optional[str] = None
    INT_save: Optional[str] = None
    WIS_score: Optional[str] = None
    WIS_mod: Optional[str] = None
    WIS_save: Optional[str] = None
    CHA_score: Optional[str] = None
    CHA_mod: Optional[str] = None
    CHA_save: Optional[str] = None

    # structured lists
    traits: List[Trait] = field(default_factory=list)
    actions: List[Action] = field(default_factory=list)
    bonus_actions: List[BonusAction] = field(default_factory=list)
    legendary_actions: List[LegendaryAction] = field(default_factory=list)


# parse the Traits section under the Traits header
def parse_traits(block):
    traits = []
    header = block.find("p", class_="monster-header", string=re.compile("Traits", re.I))
    if not header:
        return traits

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        text = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        traits.append(Trait(name=name, text=text))

    return traits


# parse the Actions section and classify each action
def parse_actions(block):
    actions = []
    header = block.find("p", class_="monster-header", string=re.compile("Actions", re.I))
    if not header:
        return actions

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect (2/Day) style usages in the action name
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the front to get the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        # classify the action type (attack, save, or other)
        if re.search(r"Attack Roll", txt):
            category = "attack"
        elif re.search(r"Saving Throw", txt):
            category = "save"
        else:
            category = "other"

        # extract attack-related fields
        m_hit = re.search(r"Attack(?: Roll)?:\s*([+-]\d+)", txt)
        m_dmg = re.search(r"\(([\dd+\-\s]+)\)\s*([A-Za-z]+) damage", txt)
        m_reach = re.search(r"reach\s+(.+?ft\.)", txt)
        m_range = re.search(r"range\s+(.+?ft\.)", txt)

        # extract save-related fields
        m_save = re.search(r"([A-Za-z]+)\s+Saving Throw:\s*DC\s*(\d+)", txt)

        actions.append(
            Action(
                name=name,
                text=body,
                category=category,
                usage=usage,
                attack_bonus=int(m_hit.group(1)) if m_hit else None,
                damage_dice=m_dmg.group(1).strip() if m_dmg else None,
                damage_type=m_dmg.group(2).lower() if m_dmg else None,
                reach=m_reach.group(1) if m_reach else None,
                range=m_range.group(1) if m_range else None,
                save_ability=m_save.group(1) if m_save else None,
                save_dc=int(m_save.group(2)) if m_save else None,
            )
        )

    return actions


# parse the Bonus Actions section
def parse_bonus_actions(block):
    bonus = []
    header = block.find("p", class_="monster-header", string=re.compile("Bonus Actions", re.I))
    if not header:
        return bonus

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect usage tags like (2/Day) in the name
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        bonus.append(BonusAction(name=name, text=body, usage=usage))

    return bonus


# parse the Legendary Actions section
def parse_legendary_actions(block):
    legs = []
    header = block.find("p", class_="monster-header", string=re.compile("Legendary Actions", re.I))
    if not header:
        return legs

    for p in header.find_all_next("p"):
        # stay inside this stat block
        if block not in p.parents:
            break
        # stop when we hit the next section header
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        # detect usage tags if present
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        # remove the bolded name from the body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        legs.append(LegendaryAction(name=name, text=body, usage=usage))

    return legs


# parse one HTML file into a list of Monster objects
def parse_monster_file(html: str) -> List[Monster]:
    # first parse: whatever the server returned
    soup = BeautifulSoup(html, "html.parser")

    # helper to find all stat-block divs
    def find_blocks(s):
        return s.find_all(
            "div",
            class_=lambda c: isinstance(c, str) and "stat-block" in c
        )

    blocks = find_blocks(soup)

    # fallback: OneDrive or other hosts may embed the real HTML as plain text
    if not blocks:
        raw_text = soup.get_text()
        if "stat-block" in raw_text:
            # parse the inner text as HTML again
            soup = BeautifulSoup(raw_text, "html.parser")
            blocks = find_blocks(soup)

    monsters: List[Monster] = []

    # if still nothing, just return empty list
    if not blocks:
        return monsters

    # loop over each stat-block in the HTML
    for block in blocks:

        # get the monster name from the heading
        header = block.find(["h3", "h4"], class_=re.compile("heading-anchor")) or \
                 block.find(["h3", "h4"])
        if not header:
            continue

        name = header.get_text(" ", strip=True)

        # get the type line right after the header
        type_p = header.find_next("p")
        mtype = type_p.get_text(" ", strip=True) if type_p else None

        size = creature_type = alignment = None
        if mtype:
            parts = mtype.split(",", 1)
            size_type = parts[0].strip()
            alignment = parts[1].strip() if len(parts) > 1 else None

            # split size/type with support for "Small or Medium"
            words = size_type.split()
            if len(words) >= 3 and words[1].lower() == "or":
                size = " ".join(words[:3])
                creature_type = " ".join(words[3:])
            else:
                size = words[0]
                creature_type = " ".join(words[1:])

        # collect all paragraph text in the stat block
        ps = block.find_all("p")
        text_lines = [p.get_text(" ", strip=True) for p in ps]

        ac = init = hp = None

        # parse AC, Initiative, and HP lines
        for t in text_lines:
            if t.startswith("AC"):
                m_ac = re.search(r"AC\s+(\d+)", t)
                ac = int(m_ac.group(1)) if m_ac else None
                m_init = re.search(r"Initiative\s*([+-]?\d+)", t)
                init = int(m_init.group(1)) if m_init else None

            if t.startswith("HP"):
                m_hp = re.search(r"HP\s+(\d+)", t)
                hp = int(m_hp.group(1)) if m_hp else None

        # helper to extract simple key/value lines (Speed, Skills, etc.)
        def extract_line(key: str):
            for t in text_lines:
                if t.startswith(key):
                    return re.sub(rf"^{key}\s*", "", t).strip()
            return None

        speed       = extract_line("Speed")
        skills      = extract_line("Skills")
        resistances = extract_line("Resistances")
        senses      = extract_line("Senses")
        languages   = extract_line("Languages")
        cr_text     = extract_line("CR")

        pb = None
        if cr_text:
            # parse PB from CR line
            m_pb = re.search(r"PB\s*([+-]\d+)", cr_text)
            pb = m_pb.group(1) if m_pb else None
            # parse numeric CR
            m_cr = re.search(r"^([0-9]+(?:/[0-9]+)?)", cr_text)
            if m_cr:
                cr_text = m_cr.group(1)

        # parse Passive Perception and clean the Senses string
        passive = None
        if senses:
            m_pass = re.search(r"Passive Perception\s*(\d+)", senses)
            if m_pass:
                passive = int(m_pass.group(1))
                senses = re.sub(r";?\s*Passive Perception\s*\d+", "", senses).strip(" ;")
                if senses == "":
                    senses = None

        # initialize ability fields
        ability_fields = {}
        for abbr in ("STR","DEX","CON","INT","WIS","CHA"):
            ability_fields[f"{abbr}_score"] = None
            ability_fields[f"{abbr}_mod"] = None
            ability_fields[f"{abbr}_save"] = None

        # parse the physical and mental ability tables
        for tbl_class in ("physical abilities-saves", "mental abilities-saves"):
            tbl = block.find("table", class_=tbl_class)
            if tbl and tbl.tbody:
                for row in tbl.tbody.find_all("tr"):
                    th = row.find("th")
                    if not th:
                        continue
                    abbr = th.get_text(" ", strip=True).upper()
                    if abbr not in ("STR","DEX","CON","INT","WIS","CHA"):
                        continue
                    cells = row.find_all("td")
                    if len(cells) >= 3:
                        ability_fields[f"{abbr}_score"] = cells[0].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_mod"]   = cells[1].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_save"]  = cells[2].get_text(" ", strip=True)

        # parse traits, actions, bonus actions, and legendary actions
        traits            = parse_traits(block)
        actions           = parse_actions(block)
        bonus_actions     = parse_bonus_actions(block)
        legendary_actions = parse_legendary_actions(block)

        # build the Monster object
        monsters.append(
            Monster(
                name=name,
                mtype=mtype,
                size=size,
                creature_type=creature_type,
                alignment=alignment,
                ac=ac,
                initiative=init,
                hp=hp,
                speed=speed,
                skills=skills,
                resistances=resistances,
                senses=senses,
                languages=languages,
                cr=cr_text,
                pb=pb,
                passive_perception=passive,
                traits=traits,
                actions=actions,
                bonus_actions=bonus_actions,
                legendary_actions=legendary_actions,
                **ability_fields
            )
        )

    return monsters


# print a readable stat block for a single monster
def pretty_print_monster(m: Monster):
    print("=" * 70)
    print(m.name)
    if m.size or m.creature_type:
        print(f"{m.size or ''} {m.creature_type or ''}, {m.alignment or ''}".strip())
    print("-" * 70)

    print(f"AC {m.ac or '—'} | HP {m.hp or '—'} | Initiative {m.initiative or '—'}")
    print(f"Speed: {m.speed or '—'}")
    if m.skills:
        print(f"Skills: {m.skills}")
    if m.resistances:
        print(f"Resistances: {m.resistances}")

    senses_line = m.senses or "—"
    if m.passive_perception:
        senses_line += f" (Passive Perception {m.passive_perception})"
    print(f"Senses: {senses_line}")

    print(f"Languages: {m.languages or '—'}")
    cr_disp = m.cr or "—"
    if m.pb:
        cr_disp += f" (PB {m.pb})"
    print(f"CR: {cr_disp}")

    print("\nAbility Scores:")
    def score(abbr):
        sc = getattr(m, f"{abbr}_score") or "—"
        md = getattr(m, f"{abbr}_mod") or "—"
        sv = getattr(m, f"{abbr}_save") or "—"
        return f"{abbr} {sc} ({md}) Save {sv}"

    print(f"{score('STR'):28} {score('DEX')}")
    print(f"{score('CON'):28} {score('INT')}")
    print(f"{score('WIS'):28} {score('CHA')}")

    if m.traits:
        print("\nTraits:")
        for t in m.traits:
            print(f"  • {t.name}: {t.text}")

    if m.actions:
        print("\nActions:")
        for a in m.actions:
            if a.usage:
                print(f"  • {a.name} ({a.usage}): {a.text}")
            else:
                print(f"  • {a.name}: {a.text}")

    if m.bonus_actions:
        print("\nBonus Actions:")
        for b in m.bonus_actions:
            if b.usage:
                print(f"  • {b.name} ({b.usage}): {b.text}")
            else:
                print(f"  • {b.name}: {b.text}")

    if m.legendary_actions:
        print("\nLegendary Actions:")
        for l in m.legendary_actions:
            if l.usage:
                print(f"  • {l.name} ({l.usage}): {l.text}")
            else:
                print(f"  • {l.name}: {l.text}")

    print("=" * 70)


# find a monster by name in the list (case-insensitive)
def get_monster_by_name(name: str, monsters: List[Monster]) -> Optional[Monster]:
    name = name.lower().strip()
    for m in monsters:
        if m.name.lower() == name:
            return m
    return None


# promptless helper to show a monster by name
def show_monster_by_name(monsters: List[Monster], name: str):
    m = get_monster_by_name(name, monsters)
    if m:
        pretty_print_monster(m)
    else:
        print("Monster not found.")


# main exec – load from OneDrive URLs instead of local files
MONSTER_FILE_URLS = [
    # your first OneDrive file link:
    "https://1drv.ms/t/c/b4ffc266ad10f68d/IQAIPGx4pqgyQ4Wi3_0CTRMtASS9Jlqm0YNEa3E792uW0ZI?e=z7LEUa",
    # add more file links here as you create them
]

all_monsters: List[Monster] = []

for url in MONSTER_FILE_URLS:
    print(f"Downloading: {url}")
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"  -> failed with status {resp.status_code}")
        continue
    html = resp.text
    monsters_from_file = parse_monster_file(html)
    print(f"  -> parsed {len(monsters_from_file)} monsters")
    all_monsters.extend(monsters_from_file)

print(f"Total monsters loaded: {len(all_monsters)}")

# example usage (uncomment after first run to test)
# show_monster_by_name(all_monsters, "Aboleth")

Downloading: https://1drv.ms/t/c/b4ffc266ad10f68d/IQAIPGx4pqgyQ4Wi3_0CTRMtASS9Jlqm0YNEa3E792uW0ZI?e=z7LEUa
  -> parsed 0 monsters
Total monsters loaded: 0


In [4]:
print("Length of HTML:", len(html))
print("Contains 'stat-block' string?", "stat-block" in html)


Length of HTML: 322429
Contains 'stat-block' string? False


In [5]:
print(data[:5000])


NameError: name 'data' is not defined

In [7]:
# parse 2024 MM-style monster stat blocks from web URLs (no local files)

import re
import requests
from dataclasses import dataclass, field
from typing import List, Optional
from bs4 import BeautifulSoup


# data classes for monster sections
@dataclass
class Trait:
    name: str
    text: str


@dataclass
class BonusAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class LegendaryAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class Action:
    name: str
    text: str
    category: Optional[str] = None     # attack, save, other

    # attack fields
    attack_bonus: Optional[int] = None
    damage_dice: Optional[str] = None
    damage_type: Optional[str] = None
    reach: Optional[str] = None
    range: Optional[str] = None

    # save fields
    save_ability: Optional[str] = None
    save_dc: Optional[int] = None

    usage: Optional[str] = None


@dataclass
class Monster:
    name: str

    # type & alignment
    mtype: Optional[str] = None
    size: Optional[str] = None
    creature_type: Optional[str] = None
    alignment: Optional[str] = None

    # core stats
    ac: Optional[int] = None
    initiative: Optional[int] = None
    hp: Optional[int] = None
    speed: Optional[str] = None
    skills: Optional[str] = None
    resistances: Optional[str] = None
    senses: Optional[str] = None
    languages: Optional[str] = None
    cr: Optional[str] = None
    pb: Optional[str] = None
    passive_perception: Optional[int] = None

    # abilities
    STR_score: Optional[str] = None
    STR_mod: Optional[str] = None
    STR_save: Optional[str] = None
    DEX_score: Optional[str] = None
    DEX_mod: Optional[str] = None
    DEX_save: Optional[str] = None
    CON_score: Optional[str] = None
    CON_mod: Optional[str] = None
    CON_save: Optional[str] = None
    INT_score: Optional[str] = None
    INT_mod: Optional[str] = None
    INT_save: Optional[str] = None
    WIS_score: Optional[str] = None
    WIS_mod: Optional[str] = None
    WIS_save: Optional[str] = None
    CHA_score: Optional[str] = None
    CHA_mod: Optional[str] = None
    CHA_save: Optional[str] = None

    # structured lists
    traits: List[Trait] = field(default_factory=list)
    actions: List[Action] = field(default_factory=list)
    bonus_actions: List[BonusAction] = field(default_factory=list)
    legendary_actions: List[LegendaryAction] = field(default_factory=list)


# parse the Traits section under the Traits header
def parse_traits(block):
    traits = []
    header = block.find("p", class_="monster-header", string=re.compile("Traits", re.I))
    if not header:
        return traits

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        text = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        traits.append(Trait(name=name, text=text))

    return traits


# parse the Actions section and classify each action
def parse_actions(block):
    actions = []
    header = block.find("p", class_="monster-header", string=re.compile("Actions", re.I))
    if not header:
        return actions

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        if re.search(r"Attack Roll", txt):
            category = "attack"
        elif re.search(r"Saving Throw", txt):
            category = "save"
        else:
            category = "other"

        m_hit = re.search(r"Attack(?: Roll)?:\s*([+-]\d+)", txt)
        m_dmg = re.search(r"\(([\dd+\-\s]+)\)\s*([A-Za-z]+) damage", txt)
        m_reach = re.search(r"reach\s+(.+?ft\.)", txt)
        m_range = re.search(r"range\s+(.+?ft\.)", txt)

        m_save = re.search(r"([A-Za-z]+)\s+Saving Throw:\s*DC\s*(\d+)", txt)

        actions.append(
            Action(
                name=name,
                text=body,
                category=category,
                usage=usage,
                attack_bonus=int(m_hit.group(1)) if m_hit else None,
                damage_dice=m_dmg.group(1).strip() if m_dmg else None,
                damage_type=m_dmg.group(2).lower() if m_dmg else None,
                reach=m_reach.group(1) if m_reach else None,
                range=m_range.group(1) if m_range else None,
                save_ability=m_save.group(1) if m_save else None,
                save_dc=int(m_save.group(2)) if m_save else None,
            )
        )

    return actions


# parse the Bonus Actions section
def parse_bonus_actions(block):
    bonus = []
    header = block.find("p", class_="monster-header", string=re.compile("Bonus Actions", re.I))
    if not header:
        return bonus

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        bonus.append(BonusAction(name=name, text=body, usage=usage))

    return bonus


# parse the Legendary Actions section
def parse_legendary_actions(block):
    legs = []
    header = block.find("p", class_="monster-header", string=re.compile("Legendary Actions", re.I))
    if not header:
        return legs

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage = None

        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[:m_use.start()].strip()

        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        legs.append(LegendaryAction(name=name, text=body, usage=usage))

    return legs


# parse one HTML blob into a list of Monster objects
def parse_monster_file(html: str) -> List[Monster]:
    soup = BeautifulSoup(html, "html.parser")

    def find_blocks(s):
        return s.find_all("div", class_=lambda c: isinstance(c, str) and "stat-block" in c)

    blocks = find_blocks(soup)

    # fallback: if wrapped, and inner HTML is in plain text with 'stat-block'
    if not blocks:
        raw_text = soup.get_text()
        if "stat-block" in raw_text:
            soup = BeautifulSoup(raw_text, "html.parser")
            blocks = find_blocks(soup)

    monsters: List[Monster] = []
    if not blocks:
        return monsters

    for block in blocks:
        header = block.find(["h3", "h4"], class_=re.compile("heading-anchor")) or \
                 block.find(["h3", "h4"])
        if not header:
            continue

        name = header.get_text(" ", strip=True)

        type_p = header.find_next("p")
        mtype = type_p.get_text(" ", strip=True) if type_p else None

        size = creature_type = alignment = None
        if mtype:
            parts = mtype.split(",", 1)
            size_type = parts[0].strip()
            alignment = parts[1].strip() if len(parts) > 1 else None

            words = size_type.split()
            if len(words) >= 3 and words[1].lower() == "or":
                size = " ".join(words[:3])
                creature_type = " ".join(words[3:])
            else:
                size = words[0]
                creature_type = " ".join(words[1:])

        ps = block.find_all("p")
        text_lines = [p.get_text(" ", strip=True) for p in ps]

        ac = init = hp = None

        for t in text_lines:
            if t.startswith("AC"):
                m_ac = re.search(r"AC\s+(\d+)", t)
                ac = int(m_ac.group(1)) if m_ac else None
                m_init = re.search(r"Initiative\s*([+-]?\d+)", t)
                init = int(m_init.group(1)) if m_init else None

            if t.startswith("HP"):
                m_hp = re.search(r"HP\s+(\d+)", t)
                hp = int(m_hp.group(1)) if m_hp else None

        def extract_line(key: str):
            for t in text_lines:
                if t.startswith(key):
                    return re.sub(rf"^{key}\s*", "", t).strip()
            return None

        speed       = extract_line("Speed")
        skills      = extract_line("Skills")
        resistances = extract_line("Resistances")
        senses      = extract_line("Senses")
        languages   = extract_line("Languages")
        cr_text     = extract_line("CR")

        pb = None
        if cr_text:
            m_pb = re.search(r"PB\s*([+-]\d+)", cr_text)
            pb = m_pb.group(1) if m_pb else None
            m_cr = re.search(r"^([0-9]+(?:/[0-9]+)?)", cr_text)
            if m_cr:
                cr_text = m_cr.group(1)

        passive = None
        if senses:
            m_pass = re.search(r"Passive Perception\s*(\d+)", senses)
            if m_pass:
                passive = int(m_pass.group(1))
                senses = re.sub(r";?\s*Passive Perception\s*\d+", "", senses).strip(" ;")
                if senses == "":
                    senses = None

        ability_fields = {}
        for abbr in ("STR","DEX","CON","INT","WIS","CHA"):
            ability_fields[f"{abbr}_score"] = None
            ability_fields[f"{abbr}_mod"] = None
            ability_fields[f"{abbr}_save"] = None

        for tbl_class in ("physical abilities-saves", "mental abilities-saves"):
            tbl = block.find("table", class_=tbl_class)
            if tbl and tbl.tbody:
                for row in tbl.tbody.find_all("tr"):
                    th = row.find("th")
                    if not th:
                        continue
                    abbr = th.get_text(" ", strip=True).upper()
                    if abbr not in ("STR","DEX","CON","INT","WIS","CHA"):
                        continue
                    cells = row.find_all("td")
                    if len(cells) >= 3:
                        ability_fields[f"{abbr}_score"] = cells[0].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_mod"]   = cells[1].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_save"]  = cells[2].get_text(" ", strip=True)

        traits            = parse_traits(block)
        actions           = parse_actions(block)
        bonus_actions     = parse_bonus_actions(block)
        legendary_actions = parse_legendary_actions(block)

        monsters.append(
            Monster(
                name=name,
                mtype=mtype,
                size=size,
                creature_type=creature_type,
                alignment=alignment,
                ac=ac,
                initiative=init,
                hp=hp,
                speed=speed,
                skills=skills,
                resistances=resistances,
                senses=senses,
                languages=languages,
                cr=cr_text,
                pb=pb,
                passive_perception=passive,
                traits=traits,
                actions=actions,
                bonus_actions=bonus_actions,
                legendary_actions=legendary_actions,
                **ability_fields
            )
        )

    return monsters


# print a readable stat block for a single monster
def pretty_print_monster(m: Monster):
    print("=" * 70)
    print(m.name)
    if m.size or m.creature_type:
        print(f"{m.size or ''} {m.creature_type or ''}, {m.alignment or ''}".strip())
    print("-" * 70)

    print(f"AC {m.ac or '—'} | HP {m.hp or '—'} | Initiative {m.initiative or '—'}")
    print(f"Speed: {m.speed or '—'}")
    if m.skills:
        print(f"Skills: {m.skills}")
    if m.resistances:
        print(f"Resistances: {m.resistances}")

    senses_line = m.senses or "—"
    if m.passive_perception:
        senses_line += f" (Passive Perception {m.passive_perception})"
    print(f"Senses: {senses_line}")

    print(f"Languages: {m.languages or '—'}")
    cr_disp = m.cr or "—"
    if m.pb:
        cr_disp += f" (PB {m.pb})"
    print(f"CR: {cr_disp}")

    print("\nAbility Scores:")
    def score(abbr):
        sc = getattr(m, f"{abbr}_score") or "—"
        md = getattr(m, f"{abbr}_mod") or "—"
        sv = getattr(m, f"{abbr}_save") or "—"
        return f"{abbr} {sc} ({md}) Save {sv}"

    print(f"{score('STR'):28} {score('DEX')}")
    print(f"{score('CON'):28} {score('INT')}")
    print(f"{score('WIS'):28} {score('CHA')}")

    if m.traits:
        print("\nTraits:")
        for t in m.traits:
            print(f"  • {t.name}: {t.text}")

    if m.actions:
        print("\nActions:")
        for a in m.actions:
            if a.usage:
                print(f"  • {a.name} ({a.usage}): {a.text}")
            else:
                print(f"  • {a.name}: {a.text}")

    if m.bonus_actions:
        print("\nBonus Actions:")
        for b in m.bonus_actions:
            if b.usage:
                print(f"  • {b.name} ({b.usage}): {b.text}")
            else:
                print(f"  • {b.name}: {b.text}")

    if m.legendary_actions:
        print("\nLegendary Actions:")
        for l in m.legendary_actions:
            if l.usage:
                print(f"  • {l.name} ({l.usage}): {l.text}")
            else:
                print(f"  • {l.name}: {l.text}")

    print("=" * 70)


# find a monster by name in the list (case-insensitive)
def get_monster_by_name(name: str, monsters: List[Monster]) -> Optional[Monster]:
    name = name.lower().strip()
    for m in monsters:
        if m.name.lower() == name:
            return m
    return None


# promptless helper to show a monster by name
def show_monster_by_name(monsters: List[Monster], name: str):
    m = get_monster_by_name(name, monsters)
    if m:
        pretty_print_monster(m)
    else:
        print("Monster not found.")


# main exec – load from generic HTTP URLs (no local paths)
MONSTER_FILE_URLS = [
    # put your real URLs here, e.g.:
    "https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/a.txt"
    # "https://your.server.edu/path/to/mm2024/monsters_a.html",
    # "https://your.server.edu/path/to/mm2024/monsters_b.html",
]

all_monsters: List[Monster] = []

for url in MONSTER_FILE_URLS:
    print(f"downloading: {url}")
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"  -> failed with status {resp.status_code}")
        continue

    html = resp.text
    monsters_from_file = parse_monster_file(html)
    print(f"  -> parsed {len(monsters_from_file)} monsters")
    all_monsters.extend(monsters_from_file)

print(f"\nloaded {len(all_monsters)} total monsters")

# example usage (uncomment after you have data)
# show_monster_by_name(all_monsters, "Aboleth")


downloading: https://1drv.ms/t/c/b4ffc266ad10f68d/IQAIPGx4pqgyQ4Wi3_0CTRMtASS9Jlqm0YNEa3E792uW0ZI?e=j8ZGmW
  -> parsed 0 monsters

loaded 0 total monsters


In [8]:
# ========================
#   TEST CATBOX MONSTER FILE
# ========================

import requests

CATBOX_URL = "https://files.catbox.moe/k90mvo.txt"

# download the text
resp = requests.get(CATBOX_URL)
if resp.status_code != 200:
    print("Failed to download file:", resp.status_code)
else:
    html = resp.text
    print("Downloaded", len(html), "characters.")

    # parse the monsters
    monsters = parse_monster_file(html)
    print("Parsed", len(monsters), "monsters.")

    # allow lookup
    def show():
        name = input("\nEnter monster name: ").strip()
        m = get_monster_by_name(name, monsters)
        if m:
            pretty_print_monster(m)
        else:
            print("Monster not found.")

    # Example usage:
    # show()


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [9]:
# imports
import re
import requests
from dataclasses import dataclass, field
from typing import List, Optional

# data classes

@dataclass
class Trait:
    name: str
    text: str


@dataclass
class BonusAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class LegendaryAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class Action:
    name: str
    text: str
    category: Optional[str] = None     # "attack", "save", "other"
    usage: Optional[str] = None        # e.g. "2/Day"

    # attack-related fields
    attack_bonus: Optional[int] = None
    damage_dice: Optional[str] = None
    damage_type: Optional[str] = None
    reach: Optional[str] = None
    range: Optional[str] = None

    # save-related fields
    save_ability: Optional[str] = None
    save_dc: Optional[int] = None


@dataclass
class Monster:
    name: str

    # type and alignment
    mtype: Optional[str] = None
    size: Optional[str] = None
    creature_type: Optional[str] = None
    alignment: Optional[str] = None

    # core stats
    ac: Optional[int] = None
    initiative: Optional[int] = None
    hp: Optional[int] = None
    speed: Optional[str] = None
    skills: Optional[str] = None
    resistances: Optional[str] = None
    immunities: Optional[str] = None
    senses: Optional[str] = None
    languages: Optional[str] = None
    cr: Optional[str] = None
    pb: Optional[str] = None
    passive_perception: Optional[int] = None

    # abilities
    STR_score: Optional[str] = None
    STR_mod: Optional[str] = None
    STR_save: Optional[str] = None
    DEX_score: Optional[str] = None
    DEX_mod: Optional[str] = None
    DEX_save: Optional[str] = None
    CON_score: Optional[str] = None
    CON_mod: Optional[str] = None
    CON_save: Optional[str] = None
    INT_score: Optional[str] = None
    INT_mod: Optional[str] = None
    INT_save: Optional[str] = None
    WIS_score: Optional[str] = None
    WIS_mod: Optional[str] = None
    WIS_save: Optional[str] = None
    CHA_score: Optional[str] = None
    CHA_mod: Optional[str] = None
    CHA_save: Optional[str] = None

    # text sections
    traits: List[Trait] = field(default_factory=list)
    actions: List[Action] = field(default_factory=list)
    bonus_actions: List[BonusAction] = field(default_factory=list)
    legendary_actions: List[LegendaryAction] = field(default_factory=list)


# parser helpers

size_words = ("Tiny", "Small", "Medium", "Large", "Huge", "Gargantuan")


def clean_lines(text: str) -> List[str]:
    """strip whitespace from each line and keep the order"""
    return [ln.strip() for ln in text.splitlines()]


def is_size_type_line(line: str) -> bool:
    """detect lines like 'Large Aberration, Lawful Evil' or 'Medium or Large Beast, Unaligned'"""
    line = line.strip()
    return any(line.startswith(sz) for sz in size_words) and "," in line


def find_monster_starts(lines: List[str]) -> List[int]:
    """locate the starting index of each monster block by pattern"""
    starts = []
    n = len(lines)
    for i in range(n - 2):
        name = lines[i]
        type_line = lines[i + 1]
        if not name or not type_line:
            continue
        if not is_size_type_line(type_line):
            continue

        # quick window check for AC and HP near the top of the block
        window = "\n".join(lines[i:i + 20])
        if "AC " in window and "HP " in window:
            starts.append(i)
    return starts


def parse_type_line(type_line: str):
    """split the 'Large Aberration, Lawful Evil' line into parts"""
    mtype = type_line.strip()
    size = creature_type = alignment = None

    parts = mtype.split(",", 1)
    size_type = parts[0].strip()
    alignment = parts[1].strip() if len(parts) > 1 else None

    words = size_type.split()
    if len(words) >= 3 and words[1].lower() == "or":
        # handles 'Small or Medium Beast'
        size = " ".join(words[:3])
        creature_type = " ".join(words[3:])
    else:
        size = words[0]
        creature_type = " ".join(words[1:])

    return mtype, size, creature_type, alignment


def parse_abilities(lines: List[str], m: Monster):
    """look for lines like 'Str 21 +5 +5' and fill in ability fields"""
    for line in lines:
        tokens = line.split()
        if len(tokens) >= 4 and tokens[0] in ("Str", "Dex", "Con", "Int", "Wis", "Cha"):
            abbr = tokens[0].upper()
            score = tokens[1]
            mod = tokens[2]
            save = tokens[3]
            setattr(m, f"{abbr}_score", score)
            setattr(m, f"{abbr}_mod", mod)
            setattr(m, f"{abbr}_save", save)


def extract_simple_field(lines: List[str], key: str) -> Optional[str]:
    """find a line that starts with 'Key ' and return the rest"""
    prefix = key + " "
    for line in lines:
        if line.startswith(prefix):
            return line[len(key):].strip()
    return None


def parse_cr_and_pb(cr_line: Optional[str]):
    """parse 'CR 10 (XP ..., PB +4)' into cr and pb"""
    cr = None
    pb = None
    if not cr_line:
        return cr, pb
    m_cr = re.search(r"CR\s+([0-9]+(?:/[0-9]+)?)", cr_line)
    if m_cr:
        cr = m_cr.group(1)
    m_pb = re.search(r"PB\s*([+-]\d+)", cr_line)
    if m_pb:
        pb = m_pb.group(1)
    return cr, pb


def parse_senses_and_passive(senses_line: Optional[str]):
    """split Senses into main senses text and Passive Perception"""
    if not senses_line:
        return None, None
    m_pass = re.search(r"Passive Perception\s*(\d+)", senses_line)
    passive = int(m_pass.group(1)) if m_pass else None
    if m_pass:
        cleaned = re.sub(r";?\s*Passive Perception\s*\d+", "", senses_line).strip(" ;")
        if cleaned == "":
            cleaned = None
        return cleaned, passive
    return senses_line, None


def split_section_blocks(section_lines: List[str]) -> List[str]:
    """split a list of lines (section body) into blocks separated by blank lines"""
    blocks = []
    cur = []
    for line in section_lines:
        if line == "":
            if cur:
                blocks.append(" ".join(cur).strip())
                cur = []
        else:
            cur.append(line)
    if cur:
        blocks.append(" ".join(cur).strip())
    return blocks


def parse_name_and_usage(raw_name: str):
    """separate '(2/Day)' usage from an action/trait name"""
    name = raw_name.strip()
    usage = None
    m_use = re.search(r"\(([^)]+)\)\s*$", name)
    if m_use:
        usage = m_use.group(1)
        name = name[:m_use.start()].strip()
    return name, usage


def split_section(lines: List[str], label: str, stop_labels: List[str]) -> List[str]:
    """get all lines between 'label' and any of the stop_labels"""
    section = []
    inside = False
    for line in lines:
        if not inside and line == label:
            inside = True
            continue
        if inside:
            if line in stop_labels:
                break
            section.append(line)
    return section


def parse_traits_section(lines: List[str]) -> List[Trait]:
    section = split_section(lines, "Traits", ["Actions", "Bonus Actions", "Legendary Actions", "Reactions"])
    blocks = split_section_blocks(section)
    traits: List[Trait] = []
    for blk in blocks:
        m = re.match(r"([^\.]+)\.\s*(.*)", blk, re.S)
        if m:
            name = m.group(1).strip()
            text = m.group(2).strip()
        else:
            name = blk[:40]
            text = blk
        traits.append(Trait(name=name, text=text))
    return traits


def parse_actions_section(lines: List[str]) -> List[Action]:
    section = split_section(lines, "Actions", ["Bonus Actions", "Legendary Actions", "Reactions"])
    blocks = split_section_blocks(section)
    actions: List[Action] = []
    for blk in blocks:
        m = re.match(r"([^\.]+)\.\s*(.*)", blk, re.S)
        if m:
            raw_name = m.group(1).strip()
            body = m.group(2).strip()
        else:
            raw_name = blk[:40]
            body = blk

        name, usage = parse_name_and_usage(raw_name)
        txt = blk

        # classify action type
        if "Attack Roll" in txt:
            category = "attack"
        elif "Saving Throw" in txt:
            category = "save"
        else:
            category = "other"

        # basic patterns for attack/saves
        m_hit = re.search(r"Attack(?: Roll)?:\s*([+-]\d+)", txt)
        m_dmg = re.search(r"\(([\dd+\-\s]+)\)\s*([A-Za-z]+) damage", txt)
        m_reach = re.search(r"reach\s+([0-9]+(?:\s*ft\.)?)", txt)
        m_range = re.search(r"range\s+([0-9/ ]+ft\.)", txt)
        m_save = re.search(r"([A-Za-z]+)\s+Saving Throw:\s*DC\s*(\d+)", txt)

        actions.append(
            Action(
                name=name,
                text=body,
                category=category,
                usage=usage,
                attack_bonus=int(m_hit.group(1)) if m_hit else None,
                damage_dice=m_dmg.group(1).strip() if m_dmg else None,
                damage_type=m_dmg.group(2).lower() if m_dmg else None,
                reach=m_reach.group(1) if m_reach else None,
                range=m_range.group(1) if m_range else None,
                save_ability=m_save.group(1) if m_save else None,
                save_dc=int(m_save.group(2)) if m_save else None,
            )
        )
    return actions


def parse_bonus_actions_section(lines: List[str]) -> List[BonusAction]:
    section = split_section(lines, "Bonus Actions", ["Legendary Actions", "Reactions"])
    blocks = split_section_blocks(section)
    bonus: List[BonusAction] = []
    for blk in blocks:
        m = re.match(r"([^\.]+)\.\s*(.*)", blk, re.S)
        if m:
            raw_name = m.group(1).strip()
            body = m.group(2).strip()
        else:
            raw_name = blk[:40]
            body = blk

        name, usage = parse_name_and_usage(raw_name)
        bonus.append(BonusAction(name=name, text=body, usage=usage))
    return bonus


def parse_legendary_actions_section(lines: List[str]) -> List[LegendaryAction]:
    section = split_section(lines, "Legendary Actions", ["Reactions"])
    blocks = split_section_blocks(section)
    legs: List[LegendaryAction] = []
    for blk in blocks:
        if blk.startswith("Legendary Action Uses"):
            continue
        m = re.match(r"([^\.]+)\.\s*(.*)", blk, re.S)
        if m:
            raw_name = m.group(1).strip()
            body = m.group(2).strip()
        else:
            raw_name = blk[:40]
            body = blk

        name, usage = parse_name_and_usage(raw_name)
        legs.append(LegendaryAction(name=name, text=body, usage=usage))
    return legs


def parse_monster_block(block_lines: List[str]) -> Monster:
    """given the lines for exactly one monster, build a Monster object"""
    name = block_lines[0]
    type_line = block_lines[1]
    mtype, size, creature_type, alignment = parse_type_line(type_line)

    m = Monster(name=name, mtype=mtype, size=size, creature_type=creature_type, alignment=alignment)

    for line in block_lines:
        if line.startswith("AC "):
            m_ac = re.search(r"AC\s+(\d+)", line)
            m.ac = int(m_ac.group(1)) if m_ac else None
            m_init = re.search(r"Initiative\s*([+-]?\d+)", line)
            m.initiative = int(m_init.group(1)) if m_init else None
        elif line.startswith("HP "):
            m_hp = re.search(r"HP\s+(\d+)", line)
            m.hp = int(m_hp.group(1)) if m_hp else None
        elif line.startswith("Speed "):
            m.speed = line[len("Speed"):].strip()

    parse_abilities(block_lines, m)

    m.skills = extract_simple_field(block_lines, "Skills")
    m.resistances = extract_simple_field(block_lines, "Resistances")
    m.immunities = extract_simple_field(block_lines, "Immunities")

    senses_line = extract_simple_field(block_lines, "Senses")
    m.senses, m.passive_perception = parse_senses_and_passive(senses_line)

    m.languages = extract_simple_field(block_lines, "Languages")

    cr_line = None
    for line in block_lines:
        if line.startswith("CR "):
            cr_line = line
            break
    m.cr, m.pb = parse_cr_and_pb(cr_line)

    m.traits = parse_traits_section(block_lines)
    m.actions = parse_actions_section(block_lines)
    m.bonus_actions = parse_bonus_actions_section(block_lines)
    m.legendary_actions = parse_legendary_actions_section(block_lines)

    return m


def parse_page_to_monsters(text: str) -> List[Monster]:
    """top-level parser: convert a full page of monster text into a list of Monster objects"""
    lines = clean_lines(text)
    starts = find_monster_starts(lines)
    monsters: List[Monster] = []
    if not starts:
        print("no monster blocks detected in this file")
        return monsters

    # add sentinel end index
    starts.append(len(lines))

    for i in range(len(starts) - 1):
        start = starts[i]
        end = starts[i + 1]
        block_lines = [ln for ln in lines[start:end] if ln is not None]
        monsters.append(parse_monster_block(block_lines))

    return monsters


# pretty-printing and lookup

def pretty_print_monster(m: Monster):
    """print a readable stat block for one monster"""
    print("=" * 70)
    print(m.name)
    if m.size or m.creature_type:
        type_line = f"{m.size or ''} {m.creature_type or ''}".strip()
        if m.alignment:
            type_line += f", {m.alignment}"
        print(type_line)
    print("-" * 70)

    print(f"AC {m.ac or '—'} | HP {m.hp or '—'} | Initiative {m.initiative or '—'}")
    print(f"Speed: {m.speed or '—'}")

    if m.skills:
        print(f"Skills: {m.skills}")
    if m.resistances:
        print(f"Resistances: {m.resistances}")
    if m.immunities:
        print(f"Immunities: {m.immunities}")

    senses_line = m.senses or "—"
    if m.passive_perception:
        senses_line += f" (Passive Perception {m.passive_perception})"
    print(f"Senses: {senses_line}")

    print(f"Languages: {m.languages or '—'}")
    cr_disp = m.cr or "—"
    if m.pb:
        cr_disp += f" (PB {m.pb})"
    print(f"CR: {cr_disp}")

    print("\nAbility Scores:")

    def score(abbr):
        sc = getattr(m, f"{abbr}_score") or "—"
        md = getattr(m, f"{abbr}_mod") or "—"
        sv = getattr(m, f"{abbr}_save") or "—"
        return f"{abbr} {sc} ({md}) Save {sv}"

    print(f"{score('STR'):28} {score('DEX')}")
    print(f"{score('CON'):28} {score('INT')}")
    print(f"{score('WIS'):28} {score('CHA')}")

    if m.traits:
        print("\nTraits:")
        for t in m.traits:
            print(f"  • {t.name}: {t.text}")

    if m.actions:
        print("\nActions:")
        for a in m.actions:
            if a.usage:
                print(f"  • {a.name} ({a.usage}): {a.text}")
            else:
                print(f"  • {a.name}: {a.text}")

    if m.bonus_actions:
        print("\nBonus Actions:")
        for b in m.bonus_actions:
            if b.usage:
                print(f"  • {b.name} ({b.usage}): {b.text}")
            else:
                print(f"  • {b.name}: {b.text}")

    if m.legendary_actions:
        print("\nLegendary Actions:")
        for l in m.legendary_actions:
            if l.usage:
                print(f"  • {l.name} ({l.usage}): {l.text}")
            else:
                print(f"  • {l.name}: {l.text}")

    print("=" * 70)


def get_monster_by_name(name: str, monsters: List[Monster]) -> Optional[Monster]:
    """lookup a monster by name (case-insensitive)"""
    name = name.lower().strip()
    for m in monsters:
        if m.name.lower() == name:
            return m
    return None


def show_monster_by_name(monsters: List[Monster]):
    """prompt for a monster name and print its stat block"""
    name = input("enter monster name: ").strip()
    m = get_monster_by_name(name, monsters)
    if m:
        pretty_print_monster(m)
    else:
        print("monster not found.")


# main exec – download all 27 files and parse

BASE_URL = "https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/"

# 26 letters + animals.txt
FILENAMES = [f"{chr(c)}.txt" for c in range(ord("a"), ord("z") + 1)]
FILENAMES.append("animals.txt")

MONSTER_FILE_URLS = [BASE_URL + fn for fn in FILENAMES]

all_monsters: List[Monster] = []

print("downloading and parsing monster files...")

for url in MONSTER_FILE_URLS:
    print(f"\nfetching {url}")
    try:
        resp = requests.get(url)
        if resp.status_code != 200:
            print("  http error:", resp.status_code)
            continue
        text = resp.text
        print("  downloaded", len(text), "characters")
        monsters = parse_page_to_monsters(text)
        print("  parsed", len(monsters), "monsters from this file")
        all_monsters.extend(monsters)
    except Exception as e:
        print("  error downloading or parsing:", e)

print("\nfinished.")
print("total monsters loaded:", len(all_monsters))

# to use:
# show_monster_by_name(all_monsters)
# e.g.:
# pretty_print_monster(get_monster_by_name("Aboleth", all_monsters))


downloading and parsing monster files...

fetching https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/a.txt
  downloaded 257251 characters
no monster blocks detected in this file
  parsed 0 monsters from this file

fetching https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/b.txt
  downloaded 448878 characters
no monster blocks detected in this file
  parsed 0 monsters from this file

fetching https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/c.txt
  downloaded 330271 characters
no monster blocks detected in this file
  parsed 0 monsters from this file

fetching https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/d.txt
  downloaded 270277 characters
no monster blocks detected in this file
  parsed 0 monsters from this file

fetching https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/e.txt
  downloaded 183571 characters
no monster blocks detected in this file
  parsed 0 monsters from this file

fetching https://froberg5.wpcomstaging.com/

In [16]:
# imports
import re
from dataclasses import dataclass, field
from typing import List, Optional

import requests
from bs4 import BeautifulSoup


# data classes
@dataclass
class Trait:
    name: str
    text: str


@dataclass
class BonusAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class LegendaryAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class Action:
    name: str
    text: str
    category: Optional[str] = None
    usage: Optional[str] = None

    attack_bonus: Optional[int] = None
    damage_dice: Optional[str] = None
    damage_type: Optional[str] = None
    reach: Optional[str] = None
    range: Optional[str] = None

    save_ability: Optional[str] = None
    save_dc: Optional[int] = None


@dataclass
class Monster:
    name: str

    mtype: Optional[str] = None
    size: Optional[str] = None
    creature_type: Optional[str] = None
    alignment: Optional[str] = None

    ac: Optional[int] = None
    initiative: Optional[int] = None
    hp: Optional[int] = None
    speed: Optional[str] = None
    skills: Optional[str] = None
    resistances: Optional[str] = None
    immunities: Optional[str] = None
    senses: Optional[str] = None
    languages: Optional[str] = None
    cr: Optional[str] = None
    pb: Optional[str] = None
    passive_perception: Optional[int] = None

    STR_score: Optional[str] = None
    STR_mod: Optional[str] = None
    STR_save: Optional[str] = None
    DEX_score: Optional[str] = None
    DEX_mod: Optional[str] = None
    DEX_save: Optional[str] = None
    CON_score: Optional[str] = None
    CON_mod: Optional[str] = None
    CON_save: Optional[str] = None
    INT_score: Optional[str] = None
    INT_mod: Optional[str] = None
    INT_save: Optional[str] = None
    WIS_score: Optional[str] = None
    WIS_mod: Optional[str] = None
    WIS_save: Optional[str] = None
    CHA_score: Optional[str] = None
    CHA_mod: Optional[str] = None
    CHA_save: Optional[str] = None

    traits: List[Trait] = field(default_factory=list)
    actions: List[Action] = field(default_factory=list)
    bonus_actions: List[BonusAction] = field(default_factory=list)
    legendary_actions: List[LegendaryAction] = field(default_factory=list)


# parser helpers
def parse_traits(block):
    traits: List[Trait] = []

    header = block.find("p", class_="monster-header", string=re.compile("Traits", re.I))
    if not header:
        return traits

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        text = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        traits.append(Trait(name=name, text=text))

    return traits


def parse_actions(block):
    actions: List[Action] = []

    header = block.find("p", class_="monster-header", string=re.compile("Actions", re.I))
    if not header:
        return actions

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage: Optional[str] = None

        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[: m_use.start()].strip()

        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        if re.search(r"Attack Roll", txt):
            category = "attack"
        elif re.search(r"Saving Throw", txt):
            category = "save"
        else:
            category = "other"

        m_hit = re.search(r"Attack(?: Roll)?:\s*([+-]\d+)", txt)
        m_dmg = re.search(r"\(([\dd+\-\s]+)\)\s*([A-Za-z]+) damage", txt)
        m_reach = re.search(r"reach\s+([0-9]+(?:\s*ft\.)?)", txt)
        m_range = re.search(r"range\s+([0-9/ ]+ft\.)", txt)
        m_save = re.search(r"([A-Za-z]+)\s+Saving Throw:\s*DC\s*(\d+)", txt)

        actions.append(
            Action(
                name=name,
                text=body,
                category=category,
                usage=usage,
                attack_bonus=int(m_hit.group(1)) if m_hit else None,
                damage_dice=m_dmg.group(1).strip() if m_dmg else None,
                damage_type=m_dmg.group(2).lower() if m_dmg else None,
                reach=m_reach.group(1) if m_reach else None,
                range=m_range.group(1) if m_range else None,
                save_ability=m_save.group(1) if m_save else None,
                save_dc=int(m_save.group(2)) if m_save else None,
            )
        )

    return actions


def parse_bonus_actions(block):
    bonus: List[BonusAction] = []

    header = block.find("p", class_="monster-header", string=re.compile("Bonus Actions", re.I))
    if not header:
        return bonus

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage: Optional[str] = None

        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[: m_use.start()].strip()

        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        bonus.append(BonusAction(name=name, text=body, usage=usage))

    return bonus


def parse_legendary_actions(block):
    legs: List[LegendaryAction] = []

    header = block.find("p", class_="monster-header", string=re.compile("Legendary Actions", re.I))
    if not header:
        return legs

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage: Optional[str] = None

        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[: m_use.start()].strip()

        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        legs.append(LegendaryAction(name=name, text=body, usage=usage))

    return legs


# main monster parser
def parse_monster_file(html: str) -> List[Monster]:
    soup = BeautifulSoup(html, "html.parser")
    monsters: List[Monster] = []

    for block in soup.find_all("div", class_=lambda c: c and "stat-block" in c):
        # find the heading for the monster (h2/h3/h4 with heading-anchor)
        header = block.find(["h2", "h3", "h4"], class_=re.compile("heading-anchor")) or \
        block.find(["h2", "h3", "h4"])

        if not header:
            continue

        name_tag = header.select_one("a.monster-tooltip") or header.find("a")
        name = name_tag.get_text(strip=True) if name_tag else header.get_text(" ", strip=True)

        type_p = header.find_next("p")
        mtype = type_p.get_text(" ", strip=True) if type_p else None

        size: Optional[str] = None
        creature_type: Optional[str] = None
        alignment: Optional[str] = None

        if mtype:
            parts = mtype.split(",", 1)
            size_type = parts[0].strip()
            alignment = parts[1].strip() if len(parts) > 1 else None

            words = size_type.split()
            if len(words) >= 3 and words[1].lower() == "or":
                size = " ".join(words[:3])
                creature_type = " ".join(words[3:])
            else:
                size = words[0]
                creature_type = " ".join(words[1:])

        # simple keys (ac, initiative, hp, speed, etc.)
        ps = block.find_all("p")

        ac: Optional[int] = None
        init: Optional[int] = None
        hp: Optional[int] = None

        for p in ps:
            strong = p.find("strong")
            if not strong:
                continue
            label = strong.get_text(strip=True)
            txt = p.get_text(" ", strip=True)
            if label == "AC":
                m_ac = re.search(r"AC\s+(\d+)", txt)
                ac = int(m_ac.group(1)) if m_ac else None
                m_init = re.search(r"Initiative\s*([+-]?\d+)", txt)
                init = int(m_init.group(1)) if m_init else None
            elif label == "HP":
                m_hp = re.search(r"HP\s+(\d+)", txt)
                hp = int(m_hp.group(1)) if m_hp else None

        def extract_line(key: str) -> Optional[str]:
            for p in ps:
                strong = p.find("strong")
                if strong and strong.get_text(strip=True) == key:
                    full = p.get_text(" ", strip=True)
                    return re.sub(rf"^{key}\s*", "", full).strip()
            return None

        speed = extract_line("Speed")
        skills = extract_line("Skills")
        resistances = extract_line("Resistances")
        immunities = extract_line("Immunities")
        senses = extract_line("Senses")
        languages = extract_line("Languages")
        cr_text = extract_line("CR")

        pb: Optional[str] = None
        cr: Optional[str] = None

        if cr_text:
            m_pb = re.search(r"PB\s*([+-]\d+)", cr_text)
            pb = m_pb.group(1) if m_pb else None
            m_cr = re.search(r"([0-9]+(?:/[0-9]+)?)", cr_text)
            cr = m_cr.group(1) if m_cr else None

        passive: Optional[int] = None
        if senses:
            m_pass = re.search(r"Passive Perception\s*(\d+)", senses)
            if m_pass:
                passive = int(m_pass.group(1))
                senses = re.sub(r";?\s*Passive Perception\s*\d+", "", senses).strip(" ;")
                if senses == "":
                    senses = None

        # ability scores
        ability_fields = {
            f"{abbr}_{field}": None
            for abbr in ("STR", "DEX", "CON", "INT", "WIS", "CHA")
            for field in ("score", "mod", "save")
        }

        for tbl_class in ("physical abilities-saves", "mental abilities-saves"):
            tbl = block.find("table", class_=tbl_class)
            if tbl and tbl.tbody:
                for row in tbl.tbody.find_all("tr"):
                    th = row.find("th")
                    if not th:
                        continue
                    abbr = th.get_text(" ", strip=True).upper()
                    if abbr not in ("STR", "DEX", "CON", "INT", "WIS", "CHA"):
                        continue
                    cells = row.find_all("td")
                    if len(cells) >= 3:
                        ability_fields[f"{abbr}_score"] = cells[0].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_mod"] = cells[1].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_save"] = cells[2].get_text(" ", strip=True)

        # sections
        traits = parse_traits(block)
        actions = parse_actions(block)
        bonus_actions = parse_bonus_actions(block)
        legendary_actions = parse_legendary_actions(block)

        # build monster
        monster = Monster(
            name=name,
            mtype=mtype,
            size=size,
            creature_type=creature_type,
            alignment=alignment,
            ac=ac,
            initiative=init,
            hp=hp,
            speed=speed,
            skills=skills,
            resistances=resistances,
            immunities=immunities,
            senses=senses,
            languages=languages,
            cr=cr,
            pb=pb,
            passive_perception=passive,
            traits=traits,
            actions=actions,
            bonus_actions=bonus_actions,
            legendary_actions=legendary_actions,
        )

        for key, value in ability_fields.items():
            setattr(monster, key, value)

        monsters.append(monster)

    return monsters


# pretty print and lookup
def pretty_print_monster(m: Monster) -> None:
    print("=" * 70)
    print(m.name)
    if m.size or m.creature_type:
        type_line = f"{m.size or ''} {m.creature_type or ''}".strip()
        if m.alignment:
            type_line += f", {m.alignment}"
        print(type_line)
    print("-" * 70)

    print(f"AC {m.ac or '—'} | HP {m.hp or '—'} | Initiative {m.initiative or '—'}")
    print(f"Speed: {m.speed or '—'}")

    if m.skills:
        print(f"Skills: {m.skills}")
    if m.resistances:
        print(f"Resistances: {m.resistances}")
    if m.immunities:
        print(f"Immunities: {m.immunities}")

    senses_line = m.senses or "—"
    if m.passive_perception:
        senses_line += f" (Passive Perception {m.passive_perception})"
    print(f"Senses: {senses_line}")

    print(f"Languages: {m.languages or '—'}")

    cr_disp = m.cr or "—"
    if m.pb:
        cr_disp += f" (PB {m.pb})"
    print(f"CR: {cr_disp}")

    print("\nAbility Scores:")

    def score(abbr: str) -> str:
        sc = getattr(m, f"{abbr}_score") or "—"
        md = getattr(m, f"{abbr}_mod") or "—"
        sv = getattr(m, f"{abbr}_save") or "—"
        return f"{abbr} {sc} ({md}) Save {sv}"

    print(f"{score('STR'):28} {score('DEX')}")
    print(f"{score('CON'):28} {score('INT')}")
    print(f"{score('WIS'):28} {score('CHA')}")

    if m.traits:
        print("\nTraits:")
        for t in m.traits:
            print(f"  • {t.name}: {t.text}")

    if m.actions:
        print("\nActions:")
        for a in m.actions:
            if a.usage:
                print(f"  • {a.name} ({a.usage}): {a.text}")
            else:
                print(f"  • {a.name}: {a.text}")

    if m.bonus_actions:
        print("\nBonus Actions:")
        for b in m.bonus_actions:
            if b.usage:
                print(f"  • {b.name} ({b.usage}): {b.text}")
            else:
                print(f"  • {b.name}: {b.text}")

    if m.legendary_actions:
        print("\nLegendary Actions:")
        for l in m.legendary_actions:
            if l.usage:
                print(f"  • {l.name} ({l.usage}): {l.text}")
            else:
                print(f"  • {l.name}: {l.text}")

    print("=" * 70)


def get_monster_by_name(name: str, monsters: List[Monster]) -> Optional[Monster]:
    target = name.lower().strip()
    for m in monsters:
        if m.name.lower() == target:
            return m
    return None


def show_monster_by_name(monsters: List[Monster]) -> None:
    name = input("enter monster name: ").strip()
    m = get_monster_by_name(name, monsters)
    if m:
        pretty_print_monster(m)
    else:
        print("monster not found.")


# main exec
if __name__ == "__main__":
    base_url = "https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/"
    filenames = [f"{chr(c)}.txt" for c in range(ord("a"), ord("z") + 1)]
    filenames.append("animals.txt")

    urls = [base_url + fn for fn in filenames]

    all_monsters: List[Monster] = []

    for url in urls:
        try:
            print(f"downloading {url} ...")
            resp = requests.get(url, timeout=30)
            if resp.status_code != 200:
                print(f"  http {resp.status_code}, skipping.")
                continue

            html = resp.content.decode("utf-8", errors="replace")
            if "stat-block" not in html:
                print("  warning: no 'stat-block' found in this file, skipping.")
                continue

            monsters = parse_monster_file(html)
            print(f"  parsed {len(monsters)} monsters.")
            all_monsters.extend(monsters)
        except Exception as e:
            print(f"  error fetching/parsing {url}: {e}")

    print(f"total monsters loaded: {len(all_monsters)}")

    if all_monsters:
        while True:
            ans = input("show a monster? (y/n): ").strip().lower()
            if ans == "y":
                show_monster_by_name(all_monsters)
            else:
                break


downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/a.txt ...
  parsed 19 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/b.txt ...
  parsed 46 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/c.txt ...
  parsed 24 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/d.txt ...
  parsed 18 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/e.txt ...
  parsed 8 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/f.txt ...
  parsed 13 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/g.txt ...
  parsed 47 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/h.txt ...
  parsed 14 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/i.txt ...
  parsed 6 monsters.
downloading https://froberg5.wpcomstaging.com/wp-content/

show a monster? (y/n):  y
enter monster name:  xorn


Xorn
Medium Elemental, Neutral
----------------------------------------------------------------------
AC 19 | HP 84 | Initiative —
Speed: 20 ft., Burrow 20 ft.
Skills: Perception +6, Stealth +6
Immunities: Poison; Paralyzed , Petrified , Poisoned
Senses: Darkvision 60 ft., Tremorsense 60 ft. (Passive Perception 16)
Languages: Primordial (Terran)
CR: 5 (PB +3)

Ability Scores:
STR 17 (+3) Save +3          DEX 10 (+0) Save +0
CON 22 (+6) Save +6          INT 11 (+0) Save +0
WIS 10 (+0) Save +0          CHA 11 (+0) Save +0

Traits:
  • Earth Glide: The xorn can burrow through nonmagical, unworked earth and stone. While doing so, the xorn doesn’t disturb the material it moves through.
  • Treasure Sense: The xorn can pinpoint the location of precious metals and stones within 60 feet of itself.

Actions:
  • Multiattack: The xorn makes one Bite attack and three Claw attacks.
  • Bite: Melee Attack Roll: +6, reach 5 ft. Hit: 17 (4d6 + 3) Piercing damage.
  • Claw: Melee Attack Roll: +6, reac

show a monster? (y/n):  y
enter monster name:  Ankheg


Ankheg
Large Monstrosity, Unaligned
----------------------------------------------------------------------
AC 14 | HP 45 | Initiative —
Speed: 30 ft., Burrow 10 ft.
Senses: Darkvision 60 ft., Tremorsense 60 ft. (Passive Perception 11)
Languages: None
CR: 2 (PB +2)

Ability Scores:
STR 17 (+3) Save +3          DEX 11 (+0) Save +0
CON 14 (+2) Save +2          INT 1 (−5) Save −5
WIS 13 (+1) Save +1          CHA 6 (−2) Save −2

Traits:
  • Tunneler: The ankheg can burrow through solid rock at half its Burrow Speed and leaves a 10-foot-diameter tunnel in its wake.

Actions:
  • Bite: Melee Attack Roll: +5 (with Advantage if the target is Grappled by the ankheg), reach 5 ft. Hit: 10 (2d6 + 3) Slashing damage plus 3 (1d6) Acid damage. If the target is a Large or smaller creature, it has the Grappled condition (escape DC 13).
  • Acid Spray (Recharge 6): Dexterity Saving Throw: DC 12, each creature in a 30-foot-long, 5-foot-wide Line . Failure: 14 (4d6) Acid damage. Success: Half damage.


show a monster? (y/n):  n


In [20]:
# imports
import re
from dataclasses import dataclass, field
from typing import List, Optional
import random
import requests
from bs4 import BeautifulSoup


# data classes
@dataclass
class Trait:
    name: str
    text: str


@dataclass
class BonusAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class LegendaryAction:
    name: str
    text: str
    usage: Optional[str] = None


@dataclass
class Action:
    name: str
    text: str
    category: Optional[str] = None
    usage: Optional[str] = None

    attack_bonus: Optional[int] = None
    damage_dice: Optional[str] = None
    damage_type: Optional[str] = None
    reach: Optional[str] = None
    range: Optional[str] = None

    save_ability: Optional[str] = None
    save_dc: Optional[int] = None


@dataclass
class Monster:
    # monster name
    name: str

    # type and alignment
    mtype: Optional[str] = None
    size: Optional[str] = None
    creature_type: Optional[str] = None
    alignment: Optional[str] = None

    # basic stats
    ac: Optional[int] = None
    initiative: Optional[int] = None
    hp: Optional[int] = None
    speed: Optional[str] = None
    skills: Optional[str] = None
    resistances: Optional[str] = None
    immunities: Optional[str] = None
    senses: Optional[str] = None
    languages: Optional[str] = None
    cr: Optional[str] = None
    pb: Optional[str] = None
    passive_perception: Optional[int] = None

    # ability scores and saves
    STR_score: Optional[str] = None
    STR_mod: Optional[str] = None
    STR_save: Optional[str] = None
    DEX_score: Optional[str] = None
    DEX_mod: Optional[str] = None
    DEX_save: Optional[str] = None
    CON_score: Optional[str] = None
    CON_mod: Optional[str] = None
    CON_save: Optional[str] = None
    INT_score: Optional[str] = None
    INT_mod: Optional[str] = None
    INT_save: Optional[str] = None
    WIS_score: Optional[str] = None
    WIS_mod: Optional[str] = None
    WIS_save: Optional[str] = None
    CHA_score: Optional[str] = None
    CHA_mod: Optional[str] = None
    CHA_save: Optional[str] = None

    # structured sections
    traits: List[Trait] = field(default_factory=list)
    actions: List[Action] = field(default_factory=list)
    bonus_actions: List[BonusAction] = field(default_factory=list)
    legendary_actions: List[LegendaryAction] = field(default_factory=list)


# parse traits section inside statblock
def parse_traits(block) -> List[Trait]:
    traits: List[Trait] = []

    header = block.find("p", class_="monster-header", string=re.compile("Traits", re.I))
    if not header:
        return traits

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        text = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        traits.append(Trait(name=name, text=text))

    return traits


# parse actions section inside stat block
def parse_actions(block) -> List[Action]:
    actions: List[Action] = []

    header = block.find("p", class_="monster-header", string=re.compile("Actions", re.I))
    if not header:
        return actions

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage: Optional[str] = None

        # pull out usage like 2/Day
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[: m_use.start()].strip()

        # remove bolded name from action body text
        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()

        # categorize action
        if re.search(r"Attack Roll", txt):
            category = "attack"
        elif re.search(r"Saving Throw", txt):
            category = "save"
        else:
            category = "other"

        # parse attack stats
        m_hit = re.search(r"Attack(?: Roll)?:\s*([+-]\d+)", txt)
        m_dmg = re.search(r"\(([\dd+\-\s]+)\)\s*([A-Za-z]+) damage", txt)
        m_reach = re.search(r"reach\s+([0-9]+(?:\s*ft\.)?)", txt)
        m_range = re.search(r"range\s+([0-9/ ]+ft\.)", txt)

        # parse save stats
        m_save = re.search(r"([A-Za-z]+)\s+Saving Throw:\s*DC\s*(\d+)", txt)

        actions.append(
            Action(
                name=name,
                text=body,
                category=category,
                usage=usage,
                attack_bonus=int(m_hit.group(1)) if m_hit else None,
                damage_dice=m_dmg.group(1).strip() if m_dmg else None,
                damage_type=m_dmg.group(2).lower() if m_dmg else None,
                reach=m_reach.group(1) if m_reach else None,
                range=m_range.group(1) if m_range else None,
                save_ability=m_save.group(1) if m_save else None,
                save_dc=int(m_save.group(2)) if m_save else None,
            )
        )

    return actions


# parse bonus actions inside statblock
def parse_bonus_actions(block) -> List[BonusAction]:
    bonus: List[BonusAction] = []

    header = block.find("p", class_="monster-header", string=re.compile("Bonus Actions", re.I))
    if not header:
        return bonus

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage: Optional[str] = None

        # pull out usage like Recharge 5–6 or 2/Day
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[: m_use.start()].strip()

        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        bonus.append(BonusAction(name=name, text=body, usage=usage))

    return bonus


# parse legendary actions inside stat block
def parse_legendary_actions(block) -> List[LegendaryAction]:
    legs: List[LegendaryAction] = []

    header = block.find("p", class_="monster-header", string=re.compile("Legendary Actions", re.I))
    if not header:
        return legs

    for p in header.find_all_next("p"):
        if block not in p.parents:
            break
        if "monster-header" in (p.get("class") or []) and p is not header:
            break

        txt = p.get_text(" ", strip=True)
        strong = p.find("strong")
        if not strong:
            continue

        raw = strong.get_text(strip=True)
        name = raw.rstrip(".")
        usage: Optional[str] = None

        # pull out usage ie 3/Day or 4 in Lair
        m_use = re.search(r"\(([^)]+)\)$", name)
        if m_use:
            usage = m_use.group(1)
            name = name[: m_use.start()].strip()

        body = re.sub(r"^" + re.escape(raw), "", txt, count=1).strip()
        legs.append(LegendaryAction(name=name, text=body, usage=usage))

    return legs


# main monster parser
def parse_monster_file(html: str) -> List[Monster]:
    soup = BeautifulSoup(html, "html.parser")
    monsters: List[Monster] = []

    # find heading for start of monster using h2/h3/h4
    for block in soup.find_all("div", class_=lambda c: c and "stat-block" in c):
        header = block.find(["h2", "h3", "h4"], class_=re.compile("heading-anchor")) or \
                 block.find(["h2", "h3", "h4"])

        if not header:
            continue

        # get monster name from tooltip link or header text
        name_tag = header.select_one("a.monster-tooltip") or header.find("a")
        name = name_tag.get_text(strip=True) if name_tag else header.get_text(" ", strip=True)

        # get size/type/alignment data
        type_p = header.find_next("p")
        mtype = type_p.get_text(" ", strip=True) if type_p else None

        size: Optional[str] = None
        creature_type: Optional[str] = None
        alignment: Optional[str] = None

        if mtype:
            parts = mtype.split(",", 1)
            size_type = parts[0].strip()
            alignment = parts[1].strip() if len(parts) > 1 else None

            words = size_type.split()
            if len(words) >= 3 and words[1].lower() == "or":
                size = " ".join(words[:3])
                creature_type = " ".join(words[3:])
            else:
                size = words[0]
                creature_type = " ".join(words[1:])

        # read <p> tags inside stat block for core stats
        ps = block.find_all("p")

        ac: Optional[int] = None
        init: Optional[int] = None
        hp: Optional[int] = None

        for p in ps:
            strong = p.find("strong")
            if not strong:
                continue
            label = strong.get_text(strip=True)
            txt = p.get_text(" ", strip=True)
            if label == "AC":
                m_ac = re.search(r"AC\s+(\d+)", txt)
                ac = int(m_ac.group(1)) if m_ac else None
                m_init = re.search(r"Initiative\s*([+-]?\d+)", txt)
                init = int(m_init.group(1)) if m_init else None
            elif label == "HP":
                m_hp = re.search(r"HP\s+(\d+)", txt)
                hp = int(m_hp.group(1)) if m_hp else None

        # helper extracts oneliners like speed, skills, etc
        def extract_line(key: str) -> Optional[str]:
            for p in ps:
                strong = p.find("strong")
                if strong and strong.get_text(strip=True) == key:
                    full = p.get_text(" ", strip=True)
                    return re.sub(rf"^{key}\s*", "", full).strip()
            return None

        # extract core lines
        speed = extract_line("Speed")
        skills = extract_line("Skills")
        resistances = extract_line("Resistances")
        immunities = extract_line("Immunities")
        senses = extract_line("Senses")
        languages = extract_line("Languages")
        cr_text = extract_line("CR")

        pb: Optional[str] = None
        cr: Optional[str] = None

        # parse CR line for CR and PB
        if cr_text:
            m_pb = re.search(r"PB\s*([+-]\d+)", cr_text)
            pb = m_pb.group(1) if m_pb else None
            m_cr = re.search(r"([0-9]+(?:/[0-9]+)?)", cr_text)
            cr = m_cr.group(1) if m_cr else None

        # parse passive percep out of senses
        passive: Optional[int] = None
        if senses:
            m_pass = re.search(r"Passive Perception\s*(\d+)", senses)
            if m_pass:
                passive = int(m_pass.group(1))
                senses = re.sub(r";?\s*Passive Perception\s*\d+", "", senses).strip(" ;")
                if senses == "":
                    senses = None

        # set up ability fields
        ability_fields = {
            f"{abbr}_{field}": None
            for abbr in ("STR", "DEX", "CON", "INT", "WIS", "CHA")
            for field in ("score", "mod", "save")
        }

        # parse physical and mental ability tables
        for tbl_class in ("physical abilities-saves", "mental abilities-saves"):
            tbl = block.find("table", class_=tbl_class)
            if tbl and tbl.tbody:
                for row in tbl.tbody.find_all("tr"):
                    th = row.find("th")
                    if not th:
                        continue
                    abbr = th.get_text(" ", strip=True).upper()
                    if abbr not in ("STR", "DEX", "CON", "INT", "WIS", "CHA"):
                        continue
                    cells = row.find_all("td")
                    if len(cells) >= 3:
                        ability_fields[f"{abbr}_score"] = cells[0].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_mod"] = cells[1].get_text(" ", strip=True)
                        ability_fields[f"{abbr}_save"] = cells[2].get_text(" ", strip=True)

        # parse sections including traits, actions, bonus actions, legendary actions
        traits = parse_traits(block)
        actions = parse_actions(block)
        bonus_actions = parse_bonus_actions(block)
        legendary_actions = parse_legendary_actions(block)

        # build monster object
        monster = Monster(
            name=name,
            mtype=mtype,
            size=size,
            creature_type=creature_type,
            alignment=alignment,
            ac=ac,
            initiative=init,
            hp=hp,
            speed=speed,
            skills=skills,
            resistances=resistances,
            immunities=immunities,
            senses=senses,
            languages=languages,
            cr=cr,
            pb=pb,
            passive_perception=passive,
            traits=traits,
            actions=actions,
            bonus_actions=bonus_actions,
            legendary_actions=legendary_actions,
        )

        # attach ability scores and saves to monster
        for key, value in ability_fields.items():
            setattr(monster, key, value)

        monsters.append(monster)

    return monsters


# pretty print readable stat block for one monstre
def pretty_print_monster(m: Monster) -> None:
    print("=" * 70)
    print(m.name)
    if m.size or m.creature_type:
        type_line = f"{m.size or ''} {m.creature_type or ''}".strip()
        if m.alignment:
            type_line += f", {m.alignment}"
        print(type_line)
    print("-" * 70)

    print(f"AC {m.ac or '—'} | HP {m.hp or '—'} | Initiative {m.initiative or '—'}")
    print(f"Speed: {m.speed or '—'}")

    if m.skills:
        print(f"Skills: {m.skills}")
    if m.resistances:
        print(f"Resistances: {m.resistances}")
    if m.immunities:
        print(f"Immunities: {m.immunities}")

    senses_line = m.senses or "—"
    if m.passive_perception:
        senses_line += f" (Passive Perception {m.passive_perception})"
    print(f"Senses: {senses_line}")

    print(f"Languages: {m.languages or '—'}")

    cr_disp = m.cr or "—"
    if m.pb:
        cr_disp += f" (PB {m.pb})"
    print(f"CR: {cr_disp}")

    print("\nAbility Scores:")

    def score(abbr: str) -> str:
        sc = getattr(m, f"{abbr}_score") or "—"
        md = getattr(m, f"{abbr}_mod") or "—"
        sv = getattr(m, f"{abbr}_save") or "—"
        return f"{abbr} {sc} ({md}) Save {sv}"

    print(f"{score('STR'):28} {score('DEX')}")
    print(f"{score('CON'):28} {score('INT')}")
    print(f"{score('WIS'):28} {score('CHA')}")

    if m.traits:
        print("\nTraits:")
        for t in m.traits:
            print(f"  • {t.name}: {t.text}")

    if m.actions:
        print("\nActions:")
        for a in m.actions:
            if a.usage:
                print(f"  • {a.name} ({a.usage}): {a.text}")
            else:
                print(f"  • {a.name}: {a.text}")

    if m.bonus_actions:
        print("\nBonus Actions:")
        for b in m.bonus_actions:
            if b.usage:
                print(f"  • {b.name} ({b.usage}): {b.text}")
            else:
                print(f"  • {b.name}: {b.text}")

    if m.legendary_actions:
        print("\nLegendary Actions:")
        for l in m.legendary_actions:
            if l.usage:
                print(f"  • {l.name} ({l.usage}): {l.text}")
            else:
                print(f"  • {l.name}: {l.text}")

    print("=" * 70)

# find monster with case-insensitive exact name match
def get_monster_by_name(name: str, monsters: List[Monster]) -> Optional[Monster]:
    target = name.lower().strip()
    for m in monsters:
        if m.name.lower() == target:
            return m
    return None


# main exec
if __name__ == "__main__":
    # base url for stat block text files
    base_url = "https://froberg5.wpcomstaging.com/wp-content/uploads/2025/11/"

    # build list of filenames a.txt through z.txt plus animals.txt
    filenames = [f"{chr(c)}.txt" for c in range(ord("a"), ord("z") + 1)]
    filenames.append("animals.txt")

    # join base url and filenames into full urls
    urls = [base_url + fn for fn in filenames]

    # list holding all monsters from all files
    all_monsters: List[Monster] = []

    # download and parse files
    for url in urls:
        try:
            print(f"Downloading {url} ...")
            resp = requests.get(url, timeout=30)
            if resp.status_code != 200:
                print(f"  http {resp.status_code}, skipping.")
                continue

            # decode utf-8 and keep going even if bytes are weird
            html = resp.content.decode("utf-8", errors="replace")
            if "stat-block" not in html:
                print("  Warning: No 'stat-block' found in this file, skipping.")
                continue

            monsters = parse_monster_file(html)
            print(f"  Parsed {len(monsters)} monsters.")
            all_monsters.extend(monsters)
        except Exception as e:
            print(f"  Error parsing {url}: {e}")

    # final summary of how many monsters were loaded
    print(f"Total monsters loaded: {len(all_monsters)}")

    # show random sample of 10 monsters
    if all_monsters:
        print("Random sample of 10 monsters:")

        sample_size = min(10, len(all_monsters))
        sample = random.sample(all_monsters, sample_size)

        for m in sample:
            print(f"  - {m.name}")

  Parsed 19 monsters.
  Parsed 46 monsters.
  Parsed 24 monsters.
  Parsed 18 monsters.
  Parsed 8 monsters.
  Parsed 13 monsters.
  Parsed 47 monsters.
  Parsed 14 monsters.
  Parsed 6 monsters.
  Parsed 1 monsters.
  Parsed 10 monsters.
  Parsed 8 monsters.
  Parsed 35 monsters.
  Parsed 7 monsters.
  Parsed 7 monsters.
  Parsed 20 monsters.
  Parsed 3 monsters.
  Parsed 13 monsters.
  Parsed 50 monsters.
  Parsed 9 monsters.
  Parsed 3 monsters.
  Parsed 6 monsters.
  Parsed 21 monsters.
  Parsed 1 monsters.
  Parsed 5 monsters.
  Parsed 3 monsters.
  Parsed 96 monsters.
Total monsters loaded: 493
Random sample of 10 monsters:
  - Blood Hawk
  - Tarrasque
  - Banshee
  - Ancient Black Dragon
  - Young Red Dragon
  - Death Slaad
  - Bandit Crime Lord
  - Swarm of Rats
  - Troll Limb
  - Werewolf


In [21]:
import random
from collections import defaultdict
from typing import Dict, Any, List, Optional, Tuple


# ---------- dice helpers ----------

def roll_d20(rng: Optional[random.Random] = None) -> int:
    """Roll a single d20."""
    rng = rng or random
    return rng.randint(1, 20)


def roll_dice_expr(expr: str, rng: Optional[random.Random] = None, times: int = 1) -> int:
    """
    Roll damage from a dice expression like "2d6", "d12", or "1d8+4".
    `times` multiplies the number of dice (used for crits).
    """
    rng = rng or random
    if not expr:
        return 0

    expr = expr.strip().lower().replace(" ", "")
    # handle leading "d6" -> "1d6"
    if expr.startswith("d"):
        expr = "1" + expr

    # split modifier
    mod = 0
    if "+" in expr:
        dice_part, mod_part = expr.split("+", 1)
        try:
            mod = int(mod_part)
        except ValueError:
            mod = 0
    elif "-" in expr[1:]:  # allow negative mods
        dice_part, mod_part = expr.split("-", 1)
        try:
            mod = -int(mod_part)
        except ValueError:
            mod = 0
    else:
        dice_part = expr

    # parse dice
    try:
        num_str, size_str = dice_part.split("d", 1)
        num = int(num_str) if num_str else 1
        size = int(size_str)
    except Exception:
        return 0

    num *= times

    total = 0
    for _ in range(num):
        total += rng.randint(1, size)
    total += mod
    return total


# ---------- generic helpers to work with your PC / Monster objects ----------

def get_initiative_bonus(creature: Any) -> int:
    """
    Try to get an initiative bonus from a PC / Monster object.
    Looks for several common attribute names; falls back to DEX mod or 0.
    """
    for attr in ("init_bonus", "initiative_bonus", "initiative"):
        if hasattr(creature, attr):
            try:
                return int(getattr(creature, attr))
            except (TypeError, ValueError):
                pass

    # try dex modifier if we have an ability score
    dex_score = None
    for attr in ("dex", "dexterity", "DEX"):
        if hasattr(creature, attr):
            dex_score = getattr(creature, attr)
            break

    try:
        if dex_score is not None:
            dex_score = int(dex_score)
            return (dex_score - 10) // 2
    except (TypeError, ValueError):
        pass

    return 0


def get_ac(creature: Any) -> int:
    """Get armor class from a PC / Monster object."""
    for attr in ("ac", "armor_class", "AC"):
        if hasattr(creature, attr):
            try:
                return int(getattr(creature, attr))
            except (TypeError, ValueError):
                pass
    raise AttributeError("Creature is missing an AC attribute (expected ac / armor_class).")


def get_max_hp(creature: Any) -> int:
    """Get maximum HP from a PC / Monster object."""
    for attr in ("max_hp", "hp", "HP"):
        if hasattr(creature, attr):
            try:
                return int(getattr(creature, attr))
            except (TypeError, ValueError):
                pass
    raise AttributeError("Creature is missing an HP attribute (expected hp / max_hp).")


def get_resistances(creature: Any) -> List[str]:
    for attr in ("resistances", "damage_resistances"):
        if hasattr(creature, attr):
            val = getattr(creature, attr) or []
            if isinstance(val, str):
                return [v.strip().lower() for v in val.split(",") if v.strip()]
            return [str(v).strip().lower() for v in val]
    return []


def get_immunities(creature: Any) -> List[str]:
    for attr in ("immunities", "damage_immunities"):
        if hasattr(creature, attr):
            val = getattr(creature, attr) or []
            if isinstance(val, str):
                return [v.strip().lower() for v in val.split(",") if v.strip()]
            return [str(v).strip().lower() for v in val]
    return []


def damage_after_resistance(creature: Any, damage: int, damage_type: str) -> int:
    """
    Apply resistance / immunity logic. `damage_type` is the raw string from your attack,
    e.g. "slashing,magic" or "fire".
    """
    if damage <= 0:
        return 0

    damage_type = (damage_type or "").lower()
    resistances = get_resistances(creature)
    immunities = get_immunities(creature)

    # simple containment check – this is not perfect but works for most stat blocks
    if any(token and token in damage_type for token in immunities):
        return 0

    if any(token and token in damage_type for token in resistances):
        return max(damage // 2, 0)

    return damage


def get_save_mod(creature: Any, ability: str) -> int:
    """
    Try to get a saving throw modifier for STR/DEX/CON/INT/WIS/CHA.
    Looks for a 'saves' dict first, then ability score to compute mod.
    """
    if not ability:
        return 0
    ability = ability.upper()
    abbr = ability[:3]  # STR, DEX, etc.

    # 1) explicit saves dict
    if hasattr(creature, "saves"):
        saves = getattr(creature, "saves") or {}
        # try exact key, then short key
        for key in (ability, abbr, ability.title(), abbr.title()):
            if key in saves:
                try:
                    return int(saves[key])
                except (TypeError, ValueError):
                    pass

    # 2) ability score
    candidate_attrs = [abbr, abbr.lower(), ability.title(), ability.lower()]
    for attr in candidate_attrs:
        if hasattr(creature, attr):
            try:
                score = int(getattr(creature, attr))
                return (score - 10) // 2
            except (TypeError, ValueError):
                pass

    return 0


# ---------- attack selection ----------

def get_pc_attacks_for_turn(pc: Any) -> List[Any]:
    """
    Very simple default: use all attacks in pc.attacks once per turn.
    You can replace this to respect Extra Attack or preferred attack choices.
    """
    attacks = getattr(pc, "attacks", [])
    return list(attacks)


def get_monster_attacks_for_turn(monster: Any) -> List[Any]:
    """
    Default: include all monster.actions that have an attack_bonus or save_dc.
    You can tighten this to follow Multiattack rules if you want.
    """
    actions = getattr(monster, "actions", [])
    usable = []
    for a in actions:
        # heuristic: any action with an attack bonus or a save DC is "offensive"
        if getattr(a, "attack_bonus", None) is not None or getattr(a, "save_dc", None) is not None:
            usable.append(a)
    return usable


# ---------- core attack resolution ----------

def resolve_attack_roll(
    attacker: Any,
    defender: Any,
    attack: Any,
    rng: Optional[random.Random] = None,
    attacker_label: str = ""
) -> Dict[str, Any]:
    """
    Resolve a standard attack roll (d20 + to_hit vs. AC).
    For PCs we assume attributes:
        attack.to_hit (int), attack.damage_dice (str), attack.damage_bonus (int), attack.damage_type (str).
    For monsters, we assume:
        attack.attack_bonus (int), attack.damage_dice (str), attack.damage_type (str),
        and no separate damage_bonus (it's baked in the damage_dice if needed).
    """
    rng = rng or random

    # determine attack bonus field name
    to_hit = None
    if hasattr(attack, "to_hit"):
        to_hit = getattr(attack, "to_hit")
    elif hasattr(attack, "attack_bonus"):
        to_hit = getattr(attack, "attack_bonus")
    try:
        to_hit = int(to_hit)
    except (TypeError, ValueError):
        to_hit = 0

    # figure out damage expression and extra flat bonus
    damage_dice = getattr(attack, "damage_dice", "") or ""
    damage_bonus = getattr(attack, "damage_bonus", 0) or 0
    damage_type = getattr(attack, "damage_type", "") or ""

    d20 = roll_d20(rng)
    crit = d20 == 20
    auto_miss = d20 == 1

    defender_ac = get_ac(defender)

    hit = False
    if not auto_miss:
        total = d20 + to_hit
        if crit or total >= defender_ac:
            hit = True

    raw_damage = 0
    if hit:
        # crit = double the dice, same flat bonus
        times = 2 if crit else 1
        raw_damage = roll_dice_expr(damage_dice, rng=rng, times=times) + int(damage_bonus)

    final_damage = damage_after_resistance(defender, raw_damage, damage_type)

    return {
        "kind": "attack_roll",
        "attacker": attacker_label,
        "attack_name": getattr(attack, "name", "attack"),
        "d20": d20,
        "crit": crit,
        "hit": hit,
        "raw_damage": raw_damage,
        "final_damage": final_damage,
        "damage_type": damage_type,
    }


def resolve_save_attack(
    attacker: Any,
    defender: Any,
    action: Any,
    rng: Optional[random.Random] = None,
    attacker_label: str = ""
) -> Dict[str, Any]:
    """
    Resolve a save-based action: defender rolls a save vs. action.save_dc.
    We assume the Monster Action has:
        save_ability (str like 'DEX'), save_dc (int), damage_dice (str), damage_type (str),
        optional half_on_success (bool, default False).
    """
    rng = rng or random

    save_ability = getattr(action, "save_ability", None)
    save_dc = getattr(action, "save_dc", None)
    try:
        save_dc = int(save_dc)
    except (TypeError, ValueError):
        save_dc = None

    damage_dice = getattr(action, "damage_dice", "") or ""
    damage_type = getattr(action, "damage_type", "") or ""
    half_on_success = bool(getattr(action, "half_on_success", False))

    if save_dc is None or not save_ability:
        # fall back to attack-roll logic if something is missing
        return resolve_attack_roll(attacker, defender, action, rng=rng, attacker_label=attacker_label)

    d20 = roll_d20(rng)
    save_mod = get_save_mod(defender, save_ability)
    total_save = d20 + save_mod
    success = total_save >= save_dc

    # treat "failure" as "hit" for hit/miss stats
    hit = not success

    base_damage = roll_dice_expr(damage_dice, rng=rng, times=1)
    if success:
        if half_on_success:
            base_damage = base_damage // 2
        else:
            base_damage = 0

    final_damage = damage_after_resistance(defender, base_damage, damage_type)

    return {
        "kind": "save",
        "attacker": attacker_label,
        "attack_name": getattr(action, "name", "save effect"),
        "d20": d20,
        "save_mod": save_mod,
        "total_save": total_save,
        "save_dc": save_dc,
        "save_ability": save_ability,
        "success": success,
        "hit": hit,
        "raw_damage": base_damage,
        "final_damage": final_damage,
        "damage_type": damage_type,
    }


def resolve_offensive_action(
    attacker: Any,
    defender: Any,
    action: Any,
    rng: Optional[random.Random] = None,
    attacker_label: str = ""
) -> Dict[str, Any]:
    """
    Dispatch based on whether the action is an attack roll or a save-based effect.
    """
    if getattr(action, "save_dc", None) is not None and getattr(action, "save_ability", None):
        return resolve_save_attack(attacker, defender, action, rng=rng, attacker_label=attacker_label)
    else:
        return resolve_attack_roll(attacker, defender, action, rng=rng, attacker_label=attacker_label)


# ---------- one full fight ----------

def roll_initiative(pc: Any, monster: Any, rng: Optional[random.Random] = None) -> Tuple[str, int, int]:
    """Return ('pc' or 'monster', pc_init, monster_init). In a tie the monster wins."""
    rng = rng or random
    pc_init = roll_d20(rng) + get_initiative_bonus(pc)
    monster_init = roll_d20(rng) + get_initiative_bonus(monster)

    if pc_init > monster_init:
        return "pc", pc_init, monster_init
    else:
        # monster wins ties
        return "monster", pc_init, monster_init


def simulate_single_fight(
    pc: Any,
    monster: Any,
    rng: Optional[random.Random] = None,
    max_rounds: int = 1000,
) -> Dict[str, Any]:
    """
    Simulate one PC vs. one Monster until one hits 0 HP.
    Returns detailed stats for this one fight.
    """
    rng = rng or random

    pc_hp = get_max_hp(pc)
    monster_hp = get_max_hp(monster)

    order, pc_init, monster_init = roll_initiative(pc, monster, rng=rng)

    rounds = 0
    winner = None

    # per-fight tallies
    hits = {"pc": 0, "monster": 0}
    misses = {"pc": 0, "monster": 0}
    damage_done = {"pc": 0, "monster": 0}
    attack_damage_by_name: Dict[Tuple[str, str], List[int]] = defaultdict(list)  # (side, attack_name) -> list of damages

    while pc_hp > 0 and monster_hp > 0 and rounds < max_rounds:
        rounds += 1

        # attacker_order = ["pc", "monster"] or vice versa
        if order == "pc":
            turn_order = ["pc", "monster"]
        else:
            turn_order = ["monster", "pc"]

        for acting_side in turn_order:
            if pc_hp <= 0 or monster_hp <= 0:
                break

            if acting_side == "pc":
                attacker = pc
                defender = monster
                defender_hp = monster_hp
                attacks = get_pc_attacks_for_turn(pc)
            else:
                attacker = monster
                defender = pc
                defender_hp = pc_hp
                attacks = get_monster_attacks_for_turn(monster)

            for action in attacks:
                if defender_hp <= 0:
                    break

                result = resolve_offensive_action(
                    attacker,
                    defender,
                    action,
                    rng=rng,
                    attacker_label=acting_side
                )

                if result["hit"]:
                    hits[acting_side] += 1
                else:
                    misses[acting_side] += 1

                dmg = int(result.get("final_damage", 0) or 0)
                if dmg > 0:
                    damage_done[acting_side] += dmg
                    attack_key = (acting_side, result["attack_name"])
                    attack_damage_by_name[attack_key].append(dmg)

                # apply damage
                defender_hp -= dmg
                if acting_side == "pc":
                    monster_hp = defender_hp
                else:
                    pc_hp = defender_hp

                if defender_hp <= 0:
                    winner = acting_side
                    break

        if pc_hp <= 0 and winner is None:
            winner = "monster"
        elif monster_hp <= 0 and winner is None:
            winner = "pc"

    if winner is None:
        # max_rounds reached – treat as draw
        winner = "draw"

    return {
        "winner": winner,
        "rounds": rounds,
        "pc_init": pc_init,
        "monster_init": monster_init,
        "hits": hits,
        "misses": misses,
        "damage_done": damage_done,
        "attack_damage_by_name": attack_damage_by_name,
    }


# ---------- Monte Carlo wrapper ----------

def simulate_many_fights(
    pc: Any,
    monster: Any,
    n_fights: int = 1000,
    seed: Optional[int] = None,
) -> Dict[str, Any]:
    """
    Run many PC vs. Monster fights and aggregate statistics.

    Returns a dict with:
        - total_fights
        - wins: {'pc': int, 'monster': int, 'draw': int}
        - init_wins: {'pc': int, 'monster': int}
        - hits: {'pc': int, 'monster': int}
        - misses: {'pc': int, 'monster': int}
        - avg_rounds: float
        - avg_damage_per_attack: { (side, attack_name): float }
    """
    rng = random.Random(seed) if seed is not None else random

    wins = {"pc": 0, "monster": 0, "draw": 0}
    init_wins = {"pc": 0, "monster": 0}
    hits = {"pc": 0, "monster": 0}
    misses = {"pc": 0, "monster": 0}
    total_rounds = 0

    damage_samples: Dict[Tuple[str, str], List[int]] = defaultdict(list)

    for _ in range(n_fights):
        result = simulate_single_fight(pc, monster, rng=rng)

        winner = result["winner"]
        wins[winner] += 1

        # initiative
        if result["pc_init"] > result["monster_init"]:
            init_wins["pc"] += 1
        else:
            init_wins["monster"] += 1  # monster wins ties

        # hits/misses
        for side in ("pc", "monster"):
            hits[side] += result["hits"].get(side, 0)
            misses[side] += result["misses"].get(side, 0)

        total_rounds += result["rounds"]

        # damage per attack
        for key, samples in result["attack_damage_by_name"].items():
            damage_samples[key].extend(samples)

    avg_damage_per_attack = {
        key: (sum(vals) / len(vals) if vals else 0.0)
        for key, vals in damage_samples.items()
    }

    avg_rounds = total_rounds / n_fights if n_fights > 0 else 0.0

    return {
        "total_fights": n_fights,
        "wins": wins,
        "init_wins": init_wins,
        "hits": hits,
        "misses": misses,
        "avg_rounds": avg_rounds,
        "avg_damage_per_attack": avg_damage_per_attack,
    }
